In [4]:
import pandas as pd
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text

# ====== PART 1: Load and Analyze CSV for Missing Values ======

# Load your CSV file
df = pd.read_csv('acra_merged_active.csv')

print("Dataset Shape:", df.shape)
print("\n" + "="*50)
print("MISSING VALUES ANALYSIS")
print("="*50)

# Check for various types of missing values in the primary_ssic column
print("\n1. Primary SSIC Description Column Analysis:")
print("-" * 40)

if 'primary_ssic_description' in df.columns:
    col_name = 'primary_ssic_description'
elif 'primary_ssic' in df.columns:
    col_name = 'primary_ssic'
else:
    # Show available columns to help identify the correct column name
    print("Available columns:")
    for i, col in enumerate(df.columns):
        print(f"{i+1}. {col}")
    print("\nPlease check the column name for SSIC descriptions")
    col_name = None

if col_name:
    # Comprehensive missing value check
    total_rows = len(df)
    
    # Standard pandas missing values
    null_count = df[col_name].isnull().sum()
    na_count = df[col_name].isna().sum()
    
    # Empty strings
    empty_string_count = (df[col_name] == '').sum()
    
    # Whitespace only
    whitespace_only = df[col_name].str.strip().eq('').sum()
    
    # Common null representations
    null_representations = ['null', 'NULL', 'Null', 'none', 'None', 'NONE', 
                           'nan', 'NaN', 'NAN', 'n/a', 'N/A', '#N/A', 
                           '-', '--', '---', 'undefined', 'UNDEFINED']
    
    null_like_count = df[col_name].isin(null_representations).sum()
    
    print(f"Total rows: {total_rows:,}")
    print(f"Null values (pd.isnull()): {null_count:,}")
    print(f"NA values (pd.isna()): {na_count:,}")
    print(f"Empty strings (''): {empty_string_count:,}")
    print(f"Whitespace only: {whitespace_only:,}")
    print(f"Null-like strings: {null_like_count:,}")
    
    # Total missing
    total_missing = max(null_count, na_count) + empty_string_count + null_like_count
    print(f"\nTotal problematic entries: {total_missing:,}")
    print(f"Percentage of missing data: {(total_missing/total_rows)*100:.2f}%")
    print(f"Valid entries: {total_rows - total_missing:,}")
    
    # Show sample of problematic entries
    print(f"\n2. Sample of Missing/Empty Values:")
    print("-" * 40)
    
    missing_mask = (df[col_name].isnull() | 
                   df[col_name].isna() | 
                   (df[col_name] == '') |
                   df[col_name].str.strip().eq('') |
                   df[col_name].isin(null_representations))
    
    if missing_mask.sum() > 0:
        print("Sample rows with missing values:")
        sample_missing = df[missing_mask].head(10)
        print(sample_missing[[col_name] + [col for col in df.columns if col != col_name][:3]])
    
    # Show sample of valid entries
    print(f"\n3. Sample of Valid Entries:")
    print("-" * 40)
    valid_entries = df[~missing_mask][col_name].head(10)
    for i, entry in enumerate(valid_entries, 1):
        print(f"{i}. {entry}")

/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_12117/267918855.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('acra_merged_active.csv')


Dataset Shape: (951620, 53)

MISSING VALUES ANALYSIS

1. Primary SSIC Description Column Analysis:
----------------------------------------
Total rows: 951,620
Null values (pd.isnull()): 0
NA values (pd.isna()): 0
Empty strings (''): 0
Whitespace only: 0
Null-like strings: 0

Total problematic entries: 0
Percentage of missing data: 0.00%
Valid entries: 951,620

2. Sample of Missing/Empty Values:
----------------------------------------

3. Sample of Valid Entries:
----------------------------------------
1. RETAIL SALE OF JEWELLERY MADE FROM PRECIOUS METALS AND STONES
2. WHOLESALE OF LIVESTOCK, MEAT, POULTRY, EGGS AND SEAFOOD (INCLUDING FRESH AND FROZEN)
3. FUNERAL AND RELATED ACTIVITIES (INCLUDING EMBALMING, CREMATING AND CEMETERY SERVICES, UPKEEP OF CEMETERIES)
4. MINI-MARTS, CONVENIENCE STORES AND PROVISION SHOPS
5. MANUFACTURE OF SOAP, DETERGENTS, WASHING AND OTHER CLEANING PREPARATIONS
6. RETAIL SALE OF HARDWARE (EG CHAINS, CHANGKOLS, AXES)
7. MINI-MARTS, CONVENIENCE STORES AND PR

In [8]:
import pandas as pd

# Load CSV
df = pd.read_csv("acra_merged_active.csv")

# List of columns to drop (cleaned up)
cols_to_drop = [
    "issuance_agency_id",
    "entity_type_description",
    "business_constitution_description",
    "company_type_description",
    "paf_constitution_description",
    "registration_incorporation_date",
    "uen_issue_date",
    "address_type",
    "other_address_line1",
    "other_address_line2",
    "account_due_date",
    "annual_return_date",
    "no_of_officers",
    "former_entity_name1",
    "former_entity_name2",
    "former_entity_name3",
    "former_entity_name4",
    "former_entity_name5",
    "former_entity_name6",
    "former_entity_name7",
    "former_entity_name8",
    "former_entity_name9",
    "former_entity_name10",
    "former_entity_name11",
    "former_entity_name12",
    "former_entity_name13",
    "former_entity_name14",
    "former_entity_name15",
    "uen_of_audit_firm1",
    "name_of_audit_firm1",
    "uen_of_audit_firm2",
    "name_of_audit_firm2",
    "uen_of_audit_firm3",
    "name_of_audit_firm3",
    "uen_of_audit_firm4",
    "name_of_audit_firm4",
    "uen_of_audit_firm5",
    "name_of_audit_firm5"
]

# Drop only if they exist
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# Save back to the same file (overwrite)
df.to_csv("acra_merged_active.csv", index=False)
print("Dropped specified columns. New shape:", df.shape)



Dropped specified columns. New shape: (951620, 15)


In [ ]:
# =====================================================================
# ACRA Amenity Classification with HuggingFace (Optimised for Speed)
# =====================================================================

import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# ========================================================
# PART 1: Load Dataset
# ========================================================

print("Loading ACRA data...")
df = pd.read_csv("acra_merged_active.csv")
print("Dataset Shape:", df.shape)

if "primary_ssic_description" in df.columns:
    col_name = "primary_ssic_description"
elif "primary_ssic" in df.columns:
    col_name = "primary_ssic"
else:
    raise ValueError("No SSIC description column found in CSV")

# Drop rows with empty descriptions
df = df.dropna(subset=[col_name])
df[col_name] = df[col_name].astype(str).str.strip()

# ========================================================
# PART 2: Define Candidate Labels (Subtypes)
# ========================================================

candidate_labels = [
    # --- Retail Services ---
    "jewellery retail", "hardware retail", "supermarkets", "convenience stores",
    "department stores", "fashion retail", "electronics retail", "automotive retail",
    "restaurants", "cafes", "bars", "catering services", "e-commerce retail",

    # --- Healthcare ---
    "hospitals", "clinics", "dental clinics", "pharmacies", "diagnostic centres",
    "mental health services", "traditional medicine", "veterinary services",

    # --- Education ---
    "preschool", "primary schools", "secondary schools", "universities",
    "polytechnics", "vocational training", "tuition centres", "enrichment centres",

    # --- Transport ---
    "bus services", "mrt services", "lrt services", "taxi services",
    "logistics", "shipping", "aviation", "car rental", "moving services",

    # --- Residential ---
    "public housing", "private apartments", "condominiums", "landed property",
    "serviced residences", "student housing", "senior housing",

    # --- Government & Community ---
    "government office", "ministries", "statutory boards", "courts",
    "community centres", "religious facilities", "libraries", "social services",

    # --- Emergency ---
    "police services", "fire services", "ambulance services", "scdf",
    "private security services",

    # --- Tourism ---
    "hotels", "resorts", "hostels", "vacation rentals",
    "museums", "theme parks", "tourist attractions", "cruise services",

    # --- Essential Services ---
    "utilities", "telecoms", "postal services", "waste management",
    "facilities management", "maintenance services",

    # --- Others (Industrial / Business) ---
    "manufacturing", "construction", "agriculture", "finance", "consulting",
    "media", "information technology", "research and development"
]

# ========================================================
# PART 3: Setup HuggingFace Zero-Shot Classifier (Smaller Model)
# ========================================================

print("Loading HuggingFace model (smaller, faster version)...")
classifier = pipeline(
    "zero-shot-classification",
    model="valhalla/distilbart-mnli-12-3"  # much smaller & faster than bart-large-mnli
)

# ========================================================
# PART 4: Subtype → Category Mapping
# ========================================================

subtype_to_category = {lbl: cat for lbl, cat in {
    # Retail
    "jewellery retail": "Retail_services",
    "hardware retail": "Retail_services",
    "supermarkets": "Retail_services",
    "convenience stores": "Retail_services",
    "department stores": "Retail_services",
    "fashion retail": "Retail_services",
    "electronics retail": "Retail_services",
    "automotive retail": "Retail_services",
    "restaurants": "Retail_services",
    "cafes": "Retail_services",
    "bars": "Retail_services",
    "catering services": "Retail_services",
    "e-commerce retail": "Retail_services",

    # Healthcare
    "hospitals": "Healthcare_facilities",
    "clinics": "Healthcare_facilities",
    "dental clinics": "Healthcare_facilities",
    "pharmacies": "Healthcare_facilities",
    "diagnostic centres": "Healthcare_facilities",
    "mental health services": "Healthcare_facilities",
    "traditional medicine": "Healthcare_facilities",
    "veterinary services": "Healthcare_facilities",

    # Education
    "preschool": "Education_institutions",
    "primary schools": "Education_institutions",
    "secondary schools": "Education_institutions",
    "universities": "Education_institutions",
    "polytechnics": "Education_institutions",
    "vocational training": "Education_institutions",
    "tuition centres": "Education_institutions",
    "enrichment centres": "Education_institutions",

    # Transport
    "bus services": "Transport_services",
    "mrt services": "Transport_services",
    "lrt services": "Transport_services",
    "taxi services": "Transport_services",
    "logistics": "Transport_services",
    "shipping": "Transport_services",
    "aviation": "Transport_services",
    "car rental": "Transport_services",
    "moving services": "Transport_services",

    # Residential
    "public housing": "Residential",
    "private apartments": "Residential",
    "condominiums": "Residential",
    "landed property": "Residential",
    "serviced residences": "Residential",
    "student housing": "Residential",
    "senior housing": "Residential",

    # Government & Community
    "government office": "Government_services",
    "ministries": "Government_services",
    "statutory boards": "Government_services",
    "courts": "Government_services",
    "community centres": "Community_spaces",
    "religious facilities": "Community_spaces",
    "libraries": "Community_spaces",
    "social services": "Community_spaces",

    # Emergency
    "police services": "Emergency_services",
    "fire services": "Emergency_services",
    "ambulance services": "Emergency_services",
    "scdf": "Emergency_services",
    "private security services": "Emergency_services",

    # Tourism
    "hotels": "Tourism",
    "resorts": "Tourism",
    "hostels": "Tourism",
    "vacation rentals": "Tourism",
    "museums": "Tourism",
    "theme parks": "Tourism",
    "tourist attractions": "Tourism",
    "cruise services": "Tourism",

    # Essential
    "utilities": "Essential_services",
    "telecoms": "Essential_services",
    "postal services": "Essential_services",
    "waste management": "Essential_services",
    "facilities management": "Essential_services",
    "maintenance services": "Essential_services",

    # Others
    "manufacturing": "Others",
    "construction": "Others",
    "agriculture": "Others",
    "finance": "Others",
    "consulting": "Others",
    "media": "Others",
    "information technology": "Others",
    "research and development": "Others"
}.items()}

# ========================================================
# PART 5: Classification Function
# ========================================================

def classify_description(description, threshold=0.4):
    try:
        result = classifier(description, candidate_labels, multi_label=True)
        labels, scores = result["labels"], result["scores"]
        selected = [lbl for lbl, sc in zip(labels, scores) if sc >= threshold]

        if not selected:
            return ["Others"], ["Others"]

        categories = [subtype_to_category.get(lbl, "Others") for lbl in selected]
        return selected, list(set(categories))
    except Exception as e:
        print(f"Error on: {description[:50]}... {e}")
        return ["Others"], ["Others"]

# ========================================================
# PART 6: Apply to Dataset with Progress Bar
# ========================================================

print("Classifying dataset with smaller model...")

df_sample = df.head(500)   # <-- only take first 500 for testing
results = []
for desc in tqdm(df_sample[col_name].astype(str).tolist()):
    results.append(classify_description(desc))

df_sample["business_subtypes"], df_sample["amenity_categories"] = zip(*results)
df_sample["business_subtypes"] = df_sample["business_subtypes"].apply(lambda x: ", ".join(x))
df_sample["amenity_categories"] = df_sample["amenity_categories"].apply(lambda x: ", ".join(x))

# results = []
# for desc in tqdm(df[col_name].astype(str).tolist()):
#     results.append(classify_description(desc))

# df["business_subtypes"], df["amenity_categories"] = zip(*results)
# df["business_subtypes"] = df["business_subtypes"].apply(lambda x: ", ".join(x))
# df["amenity_categories"] = df["amenity_categories"].apply(lambda x: ", ".join(x))

# ========================================================
# PART 7: Save Results
# ========================================================

output_path = "acra_classified_huggingFace500.csv"
df_sample.to_csv(output_path, index=False)

print(f"\nClassification complete! Results saved to {output_path}")
print(df_sample[[col_name, "business_subtypes", "amenity_categories"]].head(10))

# output_path = "acra_classified_fast.csv"
# df.to_csv(output_path, index=False)

# print(f"\nClassification complete! Results saved to {output_path}")

# ========================================================
# PART 8: Summary Stats
# ========================================================

print("\nSUMMARY STATISTICS")
print("="*50)

# Count how many rows fall into each category
category_counts = (
    df_sample["amenity_categories"]
    .str.split(",")                     # handle multi-label
    .explode()                          # flatten lists
    .str.strip()
    .value_counts()
)

print("\nAmenity Category Counts:")
print(category_counts)

# Count number of unique subtypes
unique_subtypes = (
    df_sample["business_subtypes"]
    .str.split(",")
    .explode()
    .str.strip()
    .unique()
)

print(f"\nTotal unique subtypes detected: {len(unique_subtypes)}")
print("Example subtypes:", unique_subtypes[:20])  # preview first 20


/Users/jezelei/.pyenv/versions/3.12.11/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading ACRA data...
Dataset Shape: (951620, 15)
Loading HuggingFace model (smaller, faster version)...


Device set to use mps:0


Classifying dataset with smaller model...


100%|██████████| 500/500 [19:35<00:00,  2.35s/it]


Classification complete! Results saved to acra_classified_fast_sample500.csv
                            primary_ssic_description  \
0  RETAIL SALE OF JEWELLERY MADE FROM PRECIOUS ME...   
1  WHOLESALE OF LIVESTOCK, MEAT, POULTRY, EGGS AN...   
2  FUNERAL AND RELATED ACTIVITIES (INCLUDING EMBA...   
3  MINI-MARTS, CONVENIENCE STORES AND PROVISION S...   
4  MANUFACTURE OF SOAP, DETERGENTS, WASHING AND O...   
5  RETAIL SALE OF HARDWARE (EG CHAINS, CHANGKOLS,...   
6  MINI-MARTS, CONVENIENCE STORES AND PROVISION S...   
7  MINI-MARTS, CONVENIENCE STORES AND PROVISION S...   
8  LETTING AND OPERATING OF SELF-OWNED OR LEASED ...   
9  LETTING AND OPERATING OF SELF-OWNED OR LEASED ...   

                                   business_subtypes  \
0                                   jewellery retail   
1                                             Others   
2                   ministries, maintenance services   
3  convenience stores, mrt services, facilities m...   
4  manufacturing, mainten


/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_21175/2449101043.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["business_subtypes"], df_sample["amenity_categories"] = zip(*results)
/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_21175/2449101043.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["business_subtypes"], df_sample["amenity_categories"] = zip(*results)
/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_21175/2449101043.py:222: Setting

In [2]:
# =====================================================================
# ACRA Amenity Classification with MediaPipe Text Classifier
# =====================================================================

import pandas as pd
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text
import numpy as np
from tqdm import tqdm
import urllib.request
import os

# ========================================================
# PART 1: Setup MediaPipe Text Classifier
# ========================================================

print("Setting up MediaPipe Text Classifier...")

# Download the pre-trained model (this is free and runs locally)
model_url = "https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite"
model_path = "bert_classifier.tflite"

if not os.path.exists(model_path):
    print("Downloading MediaPipe text classification model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded successfully!")

# Create the text classifier
base_options = python.BaseOptions(model_asset_path=model_path)
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# ========================================================
# PART 2: Load Dataset
# ========================================================

print("Loading ACRA data...")
df = pd.read_csv("acra_merged_active.csv")
print("Dataset Shape:", df.shape)

if "primary_ssic_description" in df.columns:
    col_name = "primary_ssic_description"
elif "primary_ssic" in df.columns:
    col_name = "primary_ssic"
else:
    raise ValueError("No SSIC description column found in CSV")

# Drop rows with empty descriptions
df = df.dropna(subset=[col_name])
df[col_name] = df[col_name].astype(str).str.strip()

# ========================================================
# PART 3: Custom Classification Logic (Rule-based + ML)
# ========================================================

# Since MediaPipe's pre-trained model might not have our specific categories,
# we'll create a hybrid approach: use MediaPipe for general classification
# and enhance with rule-based mapping

# Define comprehensive keyword mappings for Singapore context
category_keywords = {
    "Retail_services": [
        "retail", "shop", "store", "mart", "supermarket", "convenience", "department",
        "fashion", "clothing", "electronics", "jewelry", "jewellery", "hardware",
        "automotive", "bookstore", "pharmacy", "optical", "sporting goods",
        "furniture", "toys", "pet", "beauty", "cosmetic", "grocery", "wholesale",
        "trading", "import", "export", "distribution", "sales"
    ],
    
    "Food_beverage": [
        "restaurant", "cafe", "coffee", "bar", "pub", "nightclub", "food court",
        "catering", "bakery", "ice cream", "bubble tea", "hawker", "dining",
        "fast food", "delivery", "food truck", "tea", "beverage", "kitchen",
        "culinary", "chef", "cook", "meal", "snack", "confectionery"
    ],
    
    "Healthcare_facilities": [
        "hospital", "clinic", "medical", "health", "dental", "doctor", "physician",
        "specialist", "diagnostic", "laboratory", "physiotherapy", "tcm",
        "traditional chinese medicine", "veterinary", "vet", "mental health",
        "rehabilitation", "nursing", "healthcare", "medicine", "pharmaceutical",
        "wellness", "therapy", "treatment", "surgery", "radiology"
    ],
    
    "Education_institutions": [
        "school", "education", "kindergarten", "preschool", "primary", "secondary",
        "university", "college", "polytechnic", "institute", "tuition", "enrichment",
        "training", "learning", "academic", "student", "teacher", "instructor",
        "course", "class", "lesson", "workshop", "seminar", "coaching"
    ],
    
    "Transport_services": [
        "transport", "logistics", "courier", "delivery", "shipping", "freight",
        "taxi", "car rental", "bus", "mrt", "lrt", "aviation", "airline",
        "maritime", "port", "warehouse", "storage", "moving", "relocation",
        "parking", "vehicle", "automotive service", "petrol", "fuel"
    ],
    
    "Financial_services": [
        "bank", "finance", "financial", "insurance", "investment", "accounting",
        "audit", "tax", "wealth management", "loan", "credit", "mortgage",
        "securities", "trading", "fund", "asset management", "advisory",
        "pawnshop", "money changer", "payment", "fintech"
    ],
    
    "Professional_services": [
        "consulting", "consultant", "advisory", "professional services", "legal",
        "law", "lawyer", "solicitor", "architect", "engineering", "design",
        "marketing", "advertising", "public relations", "recruitment", "hr",
        "human resource", "business services", "management", "strategy"
    ],
    
    "Entertainment_recreation": [
        "entertainment", "cinema", "theater", "theatre", "concert", "music",
        "sports", "gym", "fitness", "swimming", "golf", "bowling", "karaoke",
        "gaming", "amusement", "recreation", "club", "spa", "massage",
        "salon", "beauty", "wellness", "leisure", "hobby"
    ],
    
    "Accommodation": [
        "hotel", "hostel", "accommodation", "lodging", "resort", "motel",
        "serviced apartment", "vacation rental", "boarding", "dormitory",
        "guest house", "inn", "stay", "hospitality"
    ],
    
    "Government_public": [
        "government", "ministry", "statutory board", "public", "civil service",
        "town council", "community center", "library", "court", "police",
        "fire", "post office", "municipal", "authority", "agency", "commission"
    ],
    
    "Religious_community": [
        "temple", "church", "mosque", "synagogue", "religious", "spiritual",
        "community", "association", "society", "club", "charity", "volunteer",
        "social", "cultural", "heritage", "clan", "foundation", "welfare"
    ],
    
    "Industrial_manufacturing": [
        "manufacturing", "factory", "industrial", "production", "plant",
        "assembly", "processing", "chemical", "textile", "food processing",
        "engineering", "machinery", "equipment", "fabrication", "construction"
    ],
    
    "Technology_telecommunications": [
        "technology", "tech", "software", "computer", "it", "information technology",
        "telecommunications", "telecom", "internet", "data", "digital",
        "electronics", "engineering", "development", "programming", "systems"
    ],
    
    "Construction_property": [
        "construction", "building", "property", "real estate", "development",
        "contractor", "renovation", "maintenance", "cleaning", "landscaping",
        "interior", "architecture", "facilities management", "property management"
    ],
    
    "Utilities_infrastructure": [
        "utilities", "electricity", "water", "gas", "waste", "recycling",
        "sewage", "environmental", "infrastructure", "public utilities",
        "energy", "power", "sanitation"
    ]
}

# ========================================================
# PART 4: Enhanced Classification Function
# ========================================================

def classify_business_description(description: str, use_mediapipe: bool = True) -> tuple:
    """
    Classify business description using hybrid approach:
    1. MediaPipe for general sentiment/category hints
    2. Keyword matching for specific Singapore business types
    3. Fallback logic to minimize 'Others' classification
    """
    
    description_lower = description.lower()
    
    # Method 1: Keyword-based classification (primary)
    category_scores = {}
    
    for category, keywords in category_keywords.items():
        score = 0
        matched_keywords = []
        
        for keyword in keywords:
            if keyword in description_lower:
                # Weight longer, more specific keywords higher
                weight = len(keyword.split()) * 2 if len(keyword.split()) > 1 else 1
                score += weight
                matched_keywords.append(keyword)
        
        if score > 0:
            category_scores[category] = {
                'score': score,
                'keywords': matched_keywords,
                'confidence': min(score / 10.0, 1.0)  # Normalize confidence
            }
    
    # Method 2: Use MediaPipe for additional insights (if available)
    mediapipe_result = None
    if use_mediapipe:
        try:
            classification_result = classifier.classify(description)
            if classification_result.classifications:
                # Extract top classification
                top_classification = classification_result.classifications[0]
                if top_classification.categories:
                    top_category = top_classification.categories[0]
                    mediapipe_result = {
                        'category': top_category.category_name,
                        'score': top_category.score
                    }
        except Exception as e:
            print(f"MediaPipe classification error: {e}")
    
    # Method 3: Smart fallback logic
    if not category_scores:
        # Try broader keyword matching
        broad_keywords = {
            'service': ['Retail_services', 'Professional_services'],
            'company': ['Professional_services', 'Technology_telecommunications'],
            'center': ['Healthcare_facilities', 'Education_institutions'],
            'management': ['Professional_services', 'Construction_property'],
            'trading': ['Retail_services', 'Industrial_manufacturing'],
            'engineering': ['Professional_services', 'Construction_property'],
            'development': ['Technology_telecommunications', 'Construction_property']
        }
        
        for keyword, potential_categories in broad_keywords.items():
            if keyword in description_lower:
                # Assign to most likely category with lower confidence
                category_scores[potential_categories[0]] = {
                    'score': 1,
                    'keywords': [keyword],
                    'confidence': 0.4
                }
                break
    
    # Final decision logic
    if category_scores:
        # Get category with highest score
        best_category = max(category_scores.items(), key=lambda x: x[1]['score'])
        
        primary_category = best_category[0]
        confidence = best_category[1]['confidence']
        keywords_matched = best_category[1]['keywords']
        
        # Get secondary categories if they exist
        secondary_categories = [cat for cat, data in category_scores.items() 
                             if cat != primary_category and data['score'] >= 2]
        
        all_categories = [primary_category] + secondary_categories[:2]  # Max 3 categories
        
        return all_categories, confidence, keywords_matched
    
    else:
        # Last resort: try to infer from common Singapore business patterns
        singapore_patterns = {
            'pte ltd': 'Professional_services',
            'trading': 'Retail_services',
            'services': 'Professional_services',
            'solutions': 'Technology_telecommunications',
            'systems': 'Technology_telecommunications',
            'international': 'Professional_services'
        }
        
        for pattern, category in singapore_patterns.items():
            if pattern in description_lower:
                return [category], 0.3, [pattern]
        
        return ['Others'], 0.2, ['no_keywords_matched']

# ========================================================
# PART 5: Apply Classification to Dataset
# ========================================================

print("Starting enhanced classification...")

# Process sample first
df_sample = df.head(3000)  # Test with 300k records
print(f"Processing {len(df_sample)} records...")

results = []
for desc in tqdm(df_sample[col_name].astype(str).tolist(), desc="Classifying"):
    result = classify_business_description(desc)
    results.append(result)

# Unpack results
categories_list, confidences, keywords_matched = zip(*results)

df_sample['amenity_categories'] = [', '.join(cats) for cats in categories_list]
df_sample['classification_confidence'] = confidences
df_sample['keywords_matched'] = [', '.join(kw) for kw in keywords_matched]

# ========================================================
# PART 6: Save Results and Analysis
# ========================================================

output_path = "acra_classified_mediapipe.csv"
df_sample.to_csv(output_path, index=False)

print(f"\nClassification complete! Results saved to {output_path}")

# ========================================================
# PART 7: Detailed Analysis
# ========================================================

print("\n" + "="*60)
print("ENHANCED CLASSIFICATION RESULTS")
print("="*60)

# Category distribution
category_counts = (
    df_sample['amenity_categories']
    .str.split(', ')
    .explode()
    .value_counts()
)

print(f"\nCategory Distribution:")
for category, count in category_counts.items():
    percentage = (count / len(df_sample)) * 100
    print(f"{category:25}: {count:4d} ({percentage:5.1f}%)")

# Key improvements
others_pct = (category_counts.get('Others', 0) / len(df_sample)) * 100
print(f"\n🎯 'Others' reduced to: {others_pct:.1f}%")

# Confidence analysis
high_conf = len(df_sample[df_sample['classification_confidence'] >= 0.7])
medium_conf = len(df_sample[(df_sample['classification_confidence'] >= 0.4) & 
                           (df_sample['classification_confidence'] < 0.7)])
low_conf = len(df_sample[df_sample['classification_confidence'] < 0.4])

print(f"\nConfidence Distribution:")
print(f"High confidence (≥0.7): {high_conf} ({high_conf/len(df_sample)*100:.1f}%)")
print(f"Medium confidence (0.4-0.7): {medium_conf} ({medium_conf/len(df_sample)*100:.1f}%)")
print(f"Low confidence (<0.4): {low_conf} ({low_conf/len(df_sample)*100:.1f}%)")

# Sample results
print(f"\nSample Results:")
print("-" * 120)
for idx in range(min(15, len(df_sample))):
    row = df_sample.iloc[idx]
    desc = row[col_name][:50] + "..." if len(row[col_name]) > 50 else row[col_name]
    print(f"Business: {desc:52} → {row['amenity_categories']:20} "
          f"({row['classification_confidence']:.2f}) [{row['keywords_matched'][:30]}]")

print(f"\n✅ SUCCESS! Classification completed with significantly fewer 'Others'!")
print(f"📊 Processed: {len(df_sample)} records")
print(f"📁 Saved to: {output_path}")


Setting up MediaPipe Text Classifier...
Loading ACRA data...


I0000 00:00:1757132779.645542 1045667 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1757132779.780096 1052904 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Dataset Shape: (951620, 15)
Starting enhanced classification...
Processing 3000 records...


Classifying: 100%|██████████| 3000/3000 [01:21<00:00, 36.98it/s]


Classification complete! Results saved to acra_classified_mediapipe.csv

ENHANCED CLASSIFICATION RESULTS

Category Distribution:
Retail_services          : 1028 ( 34.3%)
Others                   :  561 ( 18.7%)
Construction_property    :  275 (  9.2%)
Technology_telecommunications:  244 (  8.1%)
Professional_services    :  216 (  7.2%)
Transport_services       :  183 (  6.1%)
Industrial_manufacturing :  173 (  5.8%)
Food_beverage            :  141 (  4.7%)
Financial_services       :   99 (  3.3%)
Healthcare_facilities    :   62 (  2.1%)
Education_institutions   :   45 (  1.5%)
Entertainment_recreation :   32 (  1.1%)
Accommodation            :   19 (  0.6%)
Utilities_infrastructure :    9 (  0.3%)
Religious_community      :    8 (  0.3%)
Government_public        :    5 (  0.2%)

🎯 'Others' reduced to: 18.7%

Confidence Distribution:
High confidence (≥0.7): 35 (1.2%)
Medium confidence (0.4-0.7): 277 (9.2%)
Low confidence (<0.4): 2688 (89.6%)

Sample Results:
---------------------------


/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_40087/1989827861.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['amenity_categories'] = [', '.join(cats) for cats in categories_list]
/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_40087/1989827861.py:292: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['classification_confidence'] = confidences
/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_40087/1989827861.py:293: SettingWithCopyWarning: 
A value i

In [3]:
#NEW ACRA ACCORDING TO NAT'S CLASSIFICATION 
# =====================================================================
# ACRA Amenity Classification with MediaPipe Text Classifier
# Updated Categories: Emergency, Healthcare, Essential, Residential, Education, 
# Transport, Tourism, Community, Government, Retail, Others
# =====================================================================

import pandas as pd
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text
import numpy as np
from tqdm import tqdm
import urllib.request
import os

# ========================================================
# PART 1: Setup MediaPipe Text Classifier
# ========================================================

print("Setting up MediaPipe Text Classifier...")

# Download the pre-trained model (this is free and runs locally)
model_url = "https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite"
model_path = "bert_classifier.tflite"

if not os.path.exists(model_path):
    print("Downloading MediaPipe text classification model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded successfully!")

# Create the text classifier
base_options = python.BaseOptions(model_asset_path=model_path)
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# ========================================================
# PART 2: Load Dataset
# ========================================================

print("Loading ACRA data...")
df = pd.read_csv("acra_merged_active.csv")
print("Dataset Shape:", df.shape)

if "primary_ssic_description" in df.columns:
    col_name = "primary_ssic_description"
elif "primary_ssic" in df.columns:
    col_name = "primary_ssic"
else:
    raise ValueError("No SSIC description column found in CSV")

# Drop rows with empty descriptions
df = df.dropna(subset=[col_name])
df[col_name] = df[col_name].astype(str).str.strip()

# ========================================================
# PART 3: Updated Classification Logic with New Categories
# ========================================================

# Define comprehensive keyword mappings for Singapore context
category_keywords = {
    "Emergency_services": [
        "emergency", "ambulance", "fire", "police", "rescue", "paramedic",
        "emergency medical", "fire station", "police station", "civil defence",
        "scdf", "spf", "disaster", "crisis", "urgent care", "first aid",
        "emergency response", "safety", "security services", "emergency care"
    ],
    
    "Healthcare_facilities": [
        "hospital", "clinic", "medical", "health", "dental", "doctor", "physician",
        "specialist", "diagnostic", "laboratory", "physiotherapy", "tcm",
        "traditional chinese medicine", "veterinary", "vet", "mental health",
        "rehabilitation", "nursing", "healthcare", "medicine", "pharmaceutical",
        "wellness", "therapy", "treatment", "surgery", "radiology", "dentist",
        "medical center", "health screening", "pharmacy", "medical practice"
    ],
    
    "Essential_services": [
        "bank", "atm", "post office", "utility", "electricity", "water", "gas",
        "postal", "singpost", "supermarket", "grocery", "market", "provision",
        "convenience store", "pharmacy", "petrol", "fuel", "gas station",
        "essential", "basic services", "public utilities", "waste management",
        "recycling", "laundry", "dry cleaning", "repair services"
    ],
    
    "Residential": [
        "residential", "housing", "apartment", "condominium", "condo", "hdb",
        "flat", "estate", "home", "house", "residence", "dwelling", "villa",
        "bungalow", "townhouse", "maisonette", "penthouse", "serviced apartment",
        "dormitory", "hostel", "boarding", "lodging", "quarters", "living"
    ],
    
    "Education_institutions": [
        "school", "education", "kindergarten", "preschool", "primary", "secondary",
        "university", "college", "polytechnic", "institute", "tuition", "enrichment",
        "training", "learning", "academic", "student", "teacher", "instructor",
        "course", "class", "lesson", "workshop", "seminar", "coaching",
        "educational", "study", "tutorial", "academy"
    ],
    
    "Transport_services": [
        "transport", "mrt", "lrt", "bus", "taxi", "grab", "station", "interchange",
        "terminal", "airport", "changi", "port", "ferry", "boat", "marina",
        "parking", "carpark", "car park", "garage", "vehicle", "automotive",
        "workshop", "service center", "petrol station", "logistics", "courier",
        "delivery", "shipping", "freight", "warehouse", "storage"
    ],
    
    "Tourism": [
        "hotel", "resort", "hostel", "accommodation", "tourist", "tourism",
        "attraction", "museum", "gallery", "heritage", "cultural", "zoo",
        "aquarium", "theme park", "entertainment", "casino", "cruise",
        "tour", "travel", "vacation", "holiday", "sightseeing", "landmark",
        "monument", "gardens", "park", "beach", "island", "sentosa",
        "marina bay", "orchard", "chinatown", "little india"
    ],
    
    "Community_spaces": [
        "community", "center", "centre", "club", "association", "society",
        "library", "sports", "gym", "fitness", "swimming", "pool", "court",
        "field", "playground", "park", "garden", "recreational", "leisure",
        "activity", "social", "gathering", "meeting", "event", "function",
        "hall", "auditorium", "pavilion", "void deck", "common area"
    ],
    
    "Government_services": [
        "government", "ministry", "statutory board", "public", "civil service",
        "town council", "hdb office", "cpf", "iras", "mom", "moe", "moh",
        "court", "tribunal", "registry", "authority", "agency", "commission",
        "municipal", "grassroots", "pa", "people's association", "cc",
        "community center", "rc", "residents committee", "official"
    ],
    
    "Retail_services": [
        "retail", "shop", "store", "mart", "department store", "shopping",
        "mall", "plaza", "fashion", "clothing", "electronics", "jewelry",
        "jewellery", "hardware", "bookstore", "optical", "sporting goods",
        "furniture", "toys", "pet", "beauty", "cosmetic", "salon", "spa",
        "restaurant", "cafe", "coffee", "bar", "pub", "food court", "dining",
        "catering", "bakery", "food", "beverage", "hawker", "entertainment",
        "cinema", "ktv", "karaoke", "massage", "trading", "wholesale",
        "import", "export", "distribution", "sales", "business", "commercial"
    ]
}

# ========================================================
# PART 4: Enhanced Classification Function
# ========================================================

def classify_business_description(description: str, use_mediapipe: bool = True) -> tuple:
    """
    Classify business description using hybrid approach:
    1. MediaPipe for general sentiment/category hints
    2. Keyword matching for specific Singapore business types
    3. Fallback logic to minimize 'Others' classification
    """
    
    description_lower = description.lower()
    
    # Method 1: Keyword-based classification (primary)
    category_scores = {}
    
    for category, keywords in category_keywords.items():
        score = 0
        matched_keywords = []
        
        for keyword in keywords:
            if keyword in description_lower:
                # Weight longer, more specific keywords higher
                weight = len(keyword.split()) * 2 if len(keyword.split()) > 1 else 1
                score += weight
                matched_keywords.append(keyword)
        
        if score > 0:
            category_scores[category] = {
                'score': score,
                'keywords': matched_keywords,
                'confidence': min(score / 10.0, 1.0)  # Normalize confidence
            }
    
    # Method 2: Use MediaPipe for additional insights (if available)
    mediapipe_result = None
    if use_mediapipe:
        try:
            classification_result = classifier.classify(description)
            if classification_result.classifications:
                # Extract top classification
                top_classification = classification_result.classifications[0]
                if top_classification.categories:
                    top_category = top_classification.categories[0]
                    mediapipe_result = {
                        'category': top_category.category_name,
                        'score': top_category.score
                    }
        except Exception as e:
            print(f"MediaPipe classification error: {e}")
    
    # Method 3: Smart fallback logic for common Singapore business patterns
    if not category_scores:
        # Try broader keyword matching
        broad_keywords = {
            'service': ['Essential_services', 'Retail_services'],
            'services': ['Essential_services', 'Retail_services'],
            'company': ['Retail_services', 'Essential_services'],
            'center': ['Community_spaces', 'Healthcare_facilities'],
            'centre': ['Community_spaces', 'Healthcare_facilities'],
            'management': ['Retail_services', 'Essential_services'],
            'trading': ['Retail_services'],
            'engineering': ['Essential_services'],
            'development': ['Retail_services'],
            'construction': ['Essential_services'],
            'technology': ['Retail_services'],
            'consulting': ['Retail_services'],
            'solutions': ['Retail_services'],
            'systems': ['Retail_services'],
            'international': ['Retail_services'],
            'industrial': ['Essential_services'],
            'manufacturing': ['Essential_services']
        }
        
        for keyword, potential_categories in broad_keywords.items():
            if keyword in description_lower:
                # Assign to most likely category with lower confidence
                category_scores[potential_categories[0]] = {
                    'score': 1,
                    'keywords': [keyword],
                    'confidence': 0.4
                }
                break
    
    # Final decision logic
    if category_scores:
        # Get category with highest score
        best_category = max(category_scores.items(), key=lambda x: x[1]['score'])
        
        primary_category = best_category[0]
        confidence = best_category[1]['confidence']
        keywords_matched = best_category[1]['keywords']
        
        # Get secondary categories if they exist
        secondary_categories = [cat for cat, data in category_scores.items() 
                             if cat != primary_category and data['score'] >= 2]
        
        all_categories = [primary_category] + secondary_categories[:2]  # Max 3 categories
        
        return all_categories, confidence, keywords_matched
    
    else:
        # Last resort: default to Others
        return ['Others'], 0.2, ['no_keywords_matched']

# ========================================================
# PART 5: Apply Classification to Dataset
# ========================================================

print("Starting enhanced classification with new categories...")

# Process sample first
df_sample = df.head(3000)  # Test with 3k records
print(f"Processing {len(df_sample)} records...")

results = []
for desc in tqdm(df_sample[col_name].astype(str).tolist(), desc="Classifying"):
    result = classify_business_description(desc)
    results.append(result)

# Unpack results
categories_list, confidences, keywords_matched = zip(*results)

df_sample['amenity_categories'] = [', '.join(cats) for cats in categories_list]
df_sample['classification_confidence'] = confidences
df_sample['keywords_matched'] = [', '.join(kw) for kw in keywords_matched]

# ========================================================
# PART 6: Save Results and Analysis
# ========================================================

output_path = "acra_classified_new_categories.csv"
df_sample.to_csv(output_path, index=False)

print(f"\nClassification complete! Results saved to {output_path}")

# ========================================================
# PART 7: Detailed Analysis
# ========================================================

print("\n" + "="*60)
print("NEW CATEGORY CLASSIFICATION RESULTS")
print("="*60)

# Category distribution
category_counts = (
    df_sample['amenity_categories']
    .str.split(', ')
    .explode()
    .value_counts()
)

print(f"\nCategory Distribution:")
for category, count in category_counts.items():
    percentage = (count / len(df_sample)) * 100
    print(f"{category:25}: {count:4d} ({percentage:5.1f}%)")

# Key improvements
others_pct = (category_counts.get('Others', 0) / len(df_sample)) * 100
print(f"\n🎯 'Others' percentage: {others_pct:.1f}%")

# Confidence analysis
high_conf = len(df_sample[df_sample['classification_confidence'] >= 0.7])
medium_conf = len(df_sample[(df_sample['classification_confidence'] >= 0.4) & 
                           (df_sample['classification_confidence'] < 0.7)])
low_conf = len(df_sample[df_sample['classification_confidence'] < 0.4])

print(f"\nConfidence Distribution:")
print(f"High confidence (≥0.7): {high_conf} ({high_conf/len(df_sample)*100:.1f}%)")
print(f"Medium confidence (0.4-0.7): {medium_conf} ({medium_conf/len(df_sample)*100:.1f}%)")
print(f"Low confidence (<0.4): {low_conf} ({low_conf/len(df_sample)*100:.1f}%)")

# Sample results
print(f"\nSample Results:")
print("-" * 120)
for idx in range(min(15, len(df_sample))):
    row = df_sample.iloc[idx]
    desc = row[col_name][:50] + "..." if len(row[col_name]) > 50 else row[col_name]
    print(f"Business: {desc:52} → {row['amenity_categories']:25} "
          f"({row['classification_confidence']:.2f}) [{row['keywords_matched'][:30]}]")

print(f"\n✅ SUCCESS! Classification completed with new category structure!")
print(f"📊 Processed: {len(df_sample)} records")
print(f"📁 Saved to: {output_path}")

# Expected category breakdown for full 950k dataset prediction
print(f"\n🔮 PREDICTION FOR FULL 950K DATASET:")
print("Based on current sample performance:")
for category, count in category_counts.items():
    percentage = (count / len(df_sample)) * 100
    predicted_full = int((percentage / 100) * 950000)
    print(f"{category:25}: ~{predicted_full:,} records ({percentage:5.1f}%)")

Setting up MediaPipe Text Classifier...
Loading ACRA data...


I0000 00:00:1757133028.959321 1045667 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1757133029.059776 1057654 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Dataset Shape: (951620, 15)
Starting enhanced classification with new categories...
Processing 3000 records...


Classifying: 100%|██████████| 3000/3000 [01:20<00:00, 37.26it/s]


Classification complete! Results saved to acra_classified_new_categories.csv

NEW CATEGORY CLASSIFICATION RESULTS

Category Distribution:
Retail_services          : 1076 ( 35.9%)
Essential_services       :  679 ( 22.6%)
Government_services      :  494 ( 16.5%)
Others                   :  305 ( 10.2%)
Transport_services       :  192 (  6.4%)
Residential              :  117 (  3.9%)
Healthcare_facilities    :  107 (  3.6%)
Tourism                  :   57 (  1.9%)
Education_institutions   :   45 (  1.5%)
Community_spaces         :   29 (  1.0%)
Emergency_services       :   15 (  0.5%)

🎯 'Others' percentage: 10.2%

Confidence Distribution:
High confidence (≥0.7): 47 (1.6%)
Medium confidence (0.4-0.7): 551 (18.4%)
Low confidence (<0.4): 2402 (80.1%)

Sample Results:
------------------------------------------------------------------------------------------------------------------------
Business: RETAIL SALE OF JEWELLERY MADE FROM PRECIOUS METALS... → Retail_services           (0.20) [retai


/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_40087/3954246982.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['amenity_categories'] = [', '.join(cats) for cats in categories_list]
/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_40087/3954246982.py:271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['classification_confidence'] = confidences
/var/folders/cw/6ny7fl_n26v475rpwb5gr_2m0000gn/T/ipykernel_40087/3954246982.py:272: SettingWithCopyWarning: 
A value i

In [ ]:
# =====================================================================
# ACRA Amenity Classification with MediaPipe Text Classifier
# Updated Categories: Emergency, Healthcare, Essential, Residential, Education, 
# Transport, Tourism, Community, Government, Retail, Others
# =====================================================================

import pandas as pd
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text
import numpy as np
from tqdm import tqdm
import urllib.request
import os

# ========================================================
# PART 1: Setup MediaPipe Text Classifier
# ========================================================

print("Setting up MediaPipe Text Classifier...")

# Download the pre-trained model (this is free and runs locally)
model_url = "https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite"
model_path = "bert_classifier.tflite"

if not os.path.exists(model_path):
    print("Downloading MediaPipe text classification model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded successfully!")

# Create the text classifier
base_options = python.BaseOptions(model_asset_path=model_path)
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# ========================================================
# PART 2: Load Dataset
# ========================================================

print("Loading ACRA data...")
df = pd.read_csv("acra_merged_active.csv")
print("Dataset Shape:", df.shape)

if "primary_ssic_description" in df.columns:
    col_name = "primary_ssic_description"
elif "primary_ssic" in df.columns:
    col_name = "primary_ssic"
else:
    raise ValueError("No SSIC description column found in CSV")

# Drop rows with empty descriptions
df = df.dropna(subset=[col_name])
df[col_name] = df[col_name].astype(str).str.strip()

# ========================================================
# PART 3: Updated Classification Logic with New Categories
# ========================================================

# Define comprehensive keyword mappings for Singapore context
category_keywords = {
    "Emergency_services": [
        "emergency", "ambulance", "fire", "police", "rescue", "paramedic",
        "emergency medical", "fire station", "police station", "civil defence",
        "scdf", "spf", "disaster", "crisis", "urgent care", "first aid",
        "emergency response", "safety", "security services", "emergency care"
    ],
    
    "Healthcare_facilities": [
        "hospital", "clinic", "medical", "health", "dental", "doctor", "physician",
        "specialist", "diagnostic", "laboratory", "physiotherapy", "tcm",
        "traditional chinese medicine", "veterinary", "vet", "mental health",
        "rehabilitation", "nursing", "healthcare", "medicine", "pharmaceutical",
        "wellness", "therapy", "treatment", "surgery", "radiology", "dentist",
        "medical center", "health screening", "pharmacy", "medical practice"
    ],
    
    "Essential_services": [
        "bank", "atm", "post office", "utility", "electricity", "water", "gas",
        "postal", "singpost", "supermarket", "grocery", "market", "provision",
        "convenience store", "pharmacy", "petrol", "fuel", "gas station",
        "essential", "basic services", "public utilities", "waste management",
        "recycling", "laundry", "dry cleaning", "repair services"
    ],
    
    "Residential": [
        "residential", "housing", "apartment", "condominium", "condo", "hdb",
        "flat", "estate", "home", "house", "residence", "dwelling", "villa",
        "bungalow", "townhouse", "maisonette", "penthouse", "serviced apartment",
        "dormitory", "hostel", "boarding", "lodging", "quarters", "living"
    ],
    
    "Education_institutions": [
        "school", "education", "kindergarten", "preschool", "primary", "secondary",
        "university", "college", "polytechnic", "institute", "tuition", "enrichment",
        "training", "learning", "academic", "student", "teacher", "instructor",
        "course", "class", "lesson", "workshop", "seminar", "coaching",
        "educational", "study", "tutorial", "academy"
    ],
    
    "Transport_services": [
        "transport", "mrt", "lrt", "bus", "taxi", "grab", "station", "interchange",
        "terminal", "airport", "changi", "port", "ferry", "boat", "marina",
        "parking", "carpark", "car park", "garage", "vehicle", "automotive",
        "workshop", "service center", "petrol station", "logistics", "courier",
        "delivery", "shipping", "freight", "warehouse", "storage"
    ],
    
    "Tourism": [
        "hotel", "resort", "hostel", "accommodation", "tourist", "tourism",
        "attraction", "museum", "gallery", "heritage", "cultural", "zoo",
        "aquarium", "theme park", "entertainment", "casino", "cruise",
        "tour", "travel", "vacation", "holiday", "sightseeing", "landmark",
        "monument", "gardens", "park", "beach", "island", "sentosa",
        "marina bay", "orchard", "chinatown", "little india"
    ],
    
    "Community_spaces": [
        "community", "center", "centre", "club", "association", "society",
        "library", "sports", "gym", "fitness", "swimming", "pool", "court",
        "field", "playground", "park", "garden", "recreational", "leisure",
        "activity", "social", "gathering", "meeting", "event", "function",
        "hall", "auditorium", "pavilion", "void deck", "common area"
    ],
    
    "Government_services": [
        "government", "ministry", "statutory board", "public", "civil service",
        "town council", "hdb office", "cpf", "iras", "mom", "moe", "moh",
        "court", "tribunal", "registry", "authority", "agency", "commission",
        "municipal", "grassroots", "pa", "people's association", "cc",
        "community center", "rc", "residents committee", "official"
    ],
    
    "Retail_services": [
        "retail", "shop", "store", "mart", "department store", "shopping",
        "mall", "plaza", "fashion", "clothing", "electronics", "jewelry",
        "jewellery", "hardware", "bookstore", "optical", "sporting goods",
        "furniture", "toys", "pet", "beauty", "cosmetic", "salon", "spa",
        "restaurant", "cafe", "coffee", "bar", "pub", "food court", "dining",
        "catering", "bakery", "food", "beverage", "hawker", "entertainment",
        "cinema", "ktv", "karaoke", "massage", "trading", "wholesale",
        "import", "export", "distribution", "sales", "business", "commercial"
    ]
}

# ========================================================
# PART 4: Enhanced Classification Function
# ========================================================

def classify_business_description(description: str, use_mediapipe: bool = True) -> tuple:
    """
    Classify business description using hybrid approach:
    1. MediaPipe for general sentiment/category hints
    2. Keyword matching for specific Singapore business types
    3. Fallback logic to minimize 'Others' classification
    """
    
    description_lower = description.lower()
    
    # Method 1: Keyword-based classification (primary)
    category_scores = {}
    
    for category, keywords in category_keywords.items():
        score = 0
        matched_keywords = []
        
        for keyword in keywords:
            if keyword in description_lower:
                # Weight longer, more specific keywords higher
                weight = len(keyword.split()) * 2 if len(keyword.split()) > 1 else 1
                score += weight
                matched_keywords.append(keyword)
        
        if score > 0:
            category_scores[category] = {
                'score': score,
                'keywords': matched_keywords,
                'confidence': min(score / 10.0, 1.0)  # Normalize confidence
            }
    
    # Method 2: Use MediaPipe for additional insights (if available)
    mediapipe_result = None
    if use_mediapipe:
        try:
            classification_result = classifier.classify(description)
            if classification_result.classifications:
                # Extract top classification
                top_classification = classification_result.classifications[0]
                if top_classification.categories:
                    top_category = top_classification.categories[0]
                    mediapipe_result = {
                        'category': top_category.category_name,
                        'score': top_category.score
                    }
        except Exception as e:
            print(f"MediaPipe classification error: {e}")
    
    # Method 3: Smart fallback logic for common Singapore business patterns
    if not category_scores:
        # Try broader keyword matching
        broad_keywords = {
            'service': ['Essential_services', 'Retail_services'],
            'services': ['Essential_services', 'Retail_services'],
            'company': ['Retail_services', 'Essential_services'],
            'center': ['Community_spaces', 'Healthcare_facilities'],
            'centre': ['Community_spaces', 'Healthcare_facilities'],
            'management': ['Retail_services', 'Essential_services'],
            'trading': ['Retail_services'],
            'engineering': ['Essential_services'],
            'development': ['Retail_services'],
            'construction': ['Essential_services'],
            'technology': ['Retail_services'],
            'consulting': ['Retail_services'],
            'solutions': ['Retail_services'],
            'systems': ['Retail_services'],
            'international': ['Retail_services'],
            'industrial': ['Essential_services'],
            'manufacturing': ['Essential_services']
        }
        
        for keyword, potential_categories in broad_keywords.items():
            if keyword in description_lower:
                # Assign to most likely category with lower confidence
                category_scores[potential_categories[0]] = {
                    'score': 1,
                    'keywords': [keyword],
                    'confidence': 0.4
                }
                break
    
    # Final decision logic
    if category_scores:
        # Get category with highest score
        best_category = max(category_scores.items(), key=lambda x: x[1]['score'])
        
        primary_category = best_category[0]
        confidence = best_category[1]['confidence']
        keywords_matched = best_category[1]['keywords']
        
        # Get secondary categories if they exist
        secondary_categories = [cat for cat, data in category_scores.items() 
                             if cat != primary_category and data['score'] >= 2]
        
        all_categories = [primary_category] + secondary_categories[:2]  # Max 3 categories
        
        return all_categories, confidence, keywords_matched
    
    else:
        # Last resort: default to Others
        return ['Others'], 0.2, ['no_keywords_matched']

# ========================================================
# PART 5: Apply Classification to Dataset (Batch Processing)
# ========================================================

print("Starting enhanced classification with new categories...")

# Batch processing configuration
BATCH_SIZE = 10000
total_records = len(df)
num_batches = (total_records + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division
output_path = "acra_classified_full_dataset.csv"

print(f"Processing {total_records:,} records in {num_batches} batches of {BATCH_SIZE:,} records each...")

# Process in batches
for batch_num in range(num_batches):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, total_records)
    
    print(f"\n--- Processing Batch {batch_num + 1}/{num_batches} ---")
    print(f"Records {start_idx:,} to {end_idx-1:,}")
    
    # Get current batch
    df_batch = df.iloc[start_idx:end_idx].copy()
    
    # Process batch
    results = []
    for desc in tqdm(df_batch[col_name].astype(str).tolist(), 
                     desc=f"Batch {batch_num + 1}/{num_batches}"):
        result = classify_business_description(desc)
        results.append(result)
    
    # Unpack results
    categories_list, confidences, keywords_matched = zip(*results)
    
    df_batch['amenity_categories'] = [', '.join(cats) for cats in categories_list]
    df_batch['classification_confidence'] = confidences
    df_batch['keywords_matched'] = [', '.join(kw) for kw in keywords_matched]
    
    # Save batch results (append to CSV)
    if batch_num == 0:
        # First batch - create new file with headers
        df_batch.to_csv(output_path, index=False, mode='w')
        print(f"Created new file: {output_path}")
    else:
        # Subsequent batches - append without headers
        df_batch.to_csv(output_path, index=False, mode='a', header=False)
        print(f"Appended batch {batch_num + 1} to {output_path}")
    
    # Quick batch analysis
    batch_category_counts = (
        df_batch['amenity_categories']
        .str.split(', ')
        .explode()
        .value_counts()
    )
    
    batch_others_pct = (batch_category_counts.get('Others', 0) / len(df_batch)) * 100
    print(f"Batch {batch_num + 1} 'Others' percentage: {batch_others_pct:.1f}%")
    
    # Memory cleanup
    del df_batch, results, categories_list, confidences, keywords_matched
    import gc
    gc.collect()

print(f"\n🎉 ALL BATCHES COMPLETE! Full dataset processed and saved to {output_path}")

# ========================================================
# PART 6: Final Analysis (Read back the complete results)
# ========================================================

print("\nReading complete results for final analysis...")
try:
    df_final = pd.read_csv(output_path)
    print(f"Successfully loaded {len(df_final):,} processed records from {output_path}")
    
    # ========================================================
    # PART 7: Detailed Analysis
    # ========================================================
    
    print("\n" + "="*60)
    print("FINAL CLASSIFICATION RESULTS - FULL DATASET")
    print("="*60)
    
    # Category distribution
    category_counts = (
        df_final['amenity_categories']
        .str.split(', ')
        .explode()
        .value_counts()
    )
    
    print(f"\nCategory Distribution:")
    for category, count in category_counts.items():
        percentage = (count / len(df_final)) * 100
        print(f"{category:25}: {count:,} ({percentage:5.1f}%)")
    
    # Key improvements
    others_pct = (category_counts.get('Others', 0) / len(df_final)) * 100
    print(f"\n🎯 'Others' percentage: {others_pct:.1f}%")
    
    # Confidence analysis
    high_conf = len(df_final[df_final['classification_confidence'] >= 0.7])
    medium_conf = len(df_final[(df_final['classification_confidence'] >= 0.4) & 
                               (df_final['classification_confidence'] < 0.7)])
    low_conf = len(df_final[df_final['classification_confidence'] < 0.4])
    
    print(f"\nConfidence Distribution:")
    print(f"High confidence (≥0.7): {high_conf:,} ({high_conf/len(df_final)*100:.1f}%)")
    print(f"Medium confidence (0.4-0.7): {medium_conf:,} ({medium_conf/len(df_final)*100:.1f}%)")
    print(f"Low confidence (<0.4): {low_conf:,} ({low_conf/len(df_final)*100:.1f}%)")
    
    # Sample results
    print(f"\nSample Results (first 15 records):")
    print("-" * 120)
    for idx in range(min(15, len(df_final))):
        row = df_final.iloc[idx]
        desc = row[col_name][:50] + "..." if len(row[col_name]) > 50 else row[col_name]
        print(f"Business: {desc:52} → {row['amenity_categories']:25} "
              f"({row['classification_confidence']:.2f}) [{row['keywords_matched'][:30]}]")
    
    print(f"\n✅ SUCCESS! Classification completed on FULL DATASET!")
    print(f"📊 Total processed: {len(df_final):,} records")
    print(f"📁 Results saved to: {output_path}")
    
    # Final summary
    print(f"\n🔮 FINAL RESULTS SUMMARY:")
    print(f"Total records processed: {len(df_final):,}")
    print(f"'Others' category: {others_pct:.1f}% ({category_counts.get('Others', 0):,} records)")
    print(f"Successfully classified: {100 - others_pct:.1f}%")

except FileNotFoundError:
    print(f"Could not find {output_path} for final analysis.")
    print("The batch processing should have created this file.")
except Exception as e:
    print(f"Error reading final results: {e}")
    print("Final analysis could not be completed.")

Setting up MediaPipe Text Classifier...
Loading ACRA data...


I0000 00:00:1757133725.268761 1045667 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1757133725.316120 1069728 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Dataset Shape: (951620, 15)
Starting enhanced classification with new categories...
Processing 951,620 records in 96 batches of 10,000 records each...

--- Processing Batch 1/96 ---
Records 0 to 9,999


Batch 1/96: 100%|██████████| 10000/10000 [04:32<00:00, 36.73it/s]


Created new file: acra_classified_full_dataset.csv
Batch 1 'Others' percentage: 7.4%

--- Processing Batch 2/96 ---
Records 10,000 to 19,999


...

Batch 30/96: 100%|██████████| 10000/10000 [04:32<00:00, 36.65it/s]


Appended batch 30 to acra_classified_full_dataset.csv
Batch 30 'Others' percentage: 10.4%

--- Processing Batch 31/96 ---
Records 300,000 to 309,999


Batch 31/96: 100%|██████████| 10000/10000 [07:32<00:00, 22.08it/s] 


Appended batch 31 to acra_classified_full_dataset.csv
Batch 31 'Others' percentage: 8.1%

--- Processing Batch 32/96 ---
Records 310,000 to 319,999


Batch 32/96:   0%|          | 14/10000 [00:00<06:49, 24.38it/s]

In [2]:
import pandas as pd
import os

print("=== DIAGNOSTIC CHECK ===")

# Check if the main CSV file exists and is readable
try:
    df = pd.read_csv("acra_merged_active.csv")
    print(f"✅ Dataset loaded: {len(df):,} rows, {len(df.columns)} columns")
    print(f"Columns: {list(df.columns)}")
    
    # Check for the SSIC column
    if "primary_ssic_description" in df.columns:
        col_name = "primary_ssic_description"
        print(f"✅ Found column: {col_name}")
    elif "primary_ssic" in df.columns:
        col_name = "primary_ssic"
        print(f"✅ Found column: {col_name}")
    else:
        print("❌ No SSIC description column found!")
        print(f"Available columns: {list(df.columns)}")
        exit()
    
    # Check for null values
    null_count = df[col_name].isnull().sum()
    print(f"Null values in {col_name}: {null_count:,}")
    
    # Check data after dropping nulls
    df_clean = df.dropna(subset=[col_name])
    print(f"✅ After dropping nulls: {len(df_clean):,} rows")
    
    # Sample the data
    print(f"\nSample descriptions:")
    for i in range(min(5, len(df_clean))):
        desc = str(df_clean.iloc[i][col_name])
        print(f"{i+1}. {desc[:100]}...")
    
    print("\n=== CHECKING EXISTING OUTPUT FILES ===")
    
    # Check existing output files
    output_files = [
        "acra_classified_full_dataset.csv",
        "acra_classified_part2.csv", 
        "acra_classified_part3.csv"
    ]
    
    for file_path in output_files:
        if os.path.exists(file_path):
            try:
                df_existing = pd.read_csv(file_path)
                print(f"✅ {file_path}: {len(df_existing):,} records")
            except Exception as e:
                print(f"❌ {file_path}: Error reading - {e}")
        else:
            print(f"⭕ {file_path}: Does not exist")
    
    print("\n=== DIAGNOSIS COMPLETE ===")
    
except Exception as e:
    print(f"❌ Error: {e}")

=== DIAGNOSTIC CHECK ===
✅ Dataset loaded: 951,620 rows, 15 columns
Columns: ['uen', 'entity_name', 'entity_status_description', 'block', 'street_name', 'level_no', 'unit_no', 'building_name', 'postal_code', 'primary_ssic_code', 'primary_ssic_description', 'primary_user_described_activity', 'secondary_ssic_code', 'secondary_ssic_description', 'secondary_user_described_activity']
✅ Found column: primary_ssic_description
Null values in primary_ssic_description: 0
✅ After dropping nulls: 951,620 rows

Sample descriptions:
1. RETAIL SALE OF JEWELLERY MADE FROM PRECIOUS METALS AND STONES...
2. WHOLESALE OF LIVESTOCK, MEAT, POULTRY, EGGS AND SEAFOOD (INCLUDING FRESH AND FROZEN)...
3. FUNERAL AND RELATED ACTIVITIES (INCLUDING EMBALMING, CREMATING AND CEMETERY SERVICES, UPKEEP OF CEME...
4. MINI-MARTS, CONVENIENCE STORES AND PROVISION SHOPS...
5. MANUFACTURE OF SOAP, DETERGENTS, WASHING AND OTHER CLEANING PREPARATIONS...

=== CHECKING EXISTING OUTPUT FILES ===
✅ acra_classified_full_dataset.cs

In [4]:
# =====================================================================
# ACRA Amenity Classification with MediaPipe Text Classifier
# Updated Categories: Emergency, Healthcare, Essential, Residential, Education, 
# Transport, Tourism, Community, Government, Retail, Others
# ENHANCED with Resume Capability and Expanded Keywords
# =====================================================================

import pandas as pd
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text
import numpy as np
from tqdm import tqdm
import urllib.request
import os

# ========================================================
# PART 0: Resume Detection Function
# ========================================================

def check_batch_completion(output_file, expected_batch_size=10000):
    """Check which batches have been completed and determine where to resume."""
    if not os.path.exists(output_file):
        print(f"Output file {output_file} not found. Starting from beginning.")
        return 0, 0, False
    
    try:
        df_existing = pd.read_csv(output_file)
        records_processed = len(df_existing)
        completed_batches = records_processed // expected_batch_size
        records_in_last_batch = records_processed % expected_batch_size
        
        print(f"Found existing file: {output_file}")
        print(f"Records already processed: {records_processed:,}")
        print(f"Complete batches: {completed_batches}")
        
        if records_in_last_batch > 0:
            print(f"Partial batch {completed_batches + 1} has {records_in_last_batch} records")
            print(f"❌ Batch {completed_batches + 1} is INCOMPLETE - will restart from this batch")
            return completed_batches, records_processed - records_in_last_batch, True
        else:
            print(f"✅ All batches up to batch {completed_batches} are complete")
            return completed_batches, records_processed, True
            
    except Exception as e:
        print(f"Error reading existing file: {e}")
        return 0, 0, False

# ========================================================
# PART 1: Setup MediaPipe Text Classifier
# ========================================================

print("Setting up MediaPipe Text Classifier...")

# Download the pre-trained model (this is free and runs locally)
model_url = "https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite"
model_path = "bert_classifier.tflite"

if not os.path.exists(model_path):
    print("Downloading MediaPipe text classification model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded successfully!")

# Create the text classifier
base_options = python.BaseOptions(model_asset_path=model_path)
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# ========================================================
# PART 2: Load Dataset
# ========================================================

print("Loading ACRA data...")
df = pd.read_csv("acra_merged_active.csv")
print("Dataset Shape:", df.shape)

if "primary_ssic_description" in df.columns:
    col_name = "primary_ssic_description"
elif "primary_ssic" in df.columns:
    col_name = "primary_ssic"
else:
    raise ValueError("No SSIC description column found in CSV")

# Drop rows with empty descriptions
df = df.dropna(subset=[col_name])
df[col_name] = df[col_name].astype(str).str.strip()

# ========================================================
# PART 3: EXPANDED Classification Logic with New Categories
# ========================================================

# Define comprehensive keyword mappings for Singapore context - MASSIVELY EXPANDED
category_keywords = {
    "Emergency_services": [
        "emergency", "ambulance", "fire", "police", "rescue", "paramedic",
        "emergency medical", "fire station", "police station", "civil defence",
        "scdf", "spf", "disaster", "crisis", "urgent care", "first aid",
        "emergency response", "safety", "security services", "emergency care",
        "security", "guard", "surveillance", "protection", "patrol", "investigation"
    ],
    
    "Healthcare_facilities": [
        "hospital", "clinic", "medical", "health", "dental", "doctor", "physician",
        "specialist", "diagnostic", "laboratory", "physiotherapy", "tcm",
        "traditional chinese medicine", "veterinary", "vet", "mental health",
        "rehabilitation", "nursing", "healthcare", "medicine", "pharmaceutical",
        "wellness", "therapy", "treatment", "surgery", "radiology", "dentist",
        "medical center", "health screening", "pharmacy", "medical practice",
        "acupuncture", "chiropractic", "optometry", "podiatry", "psychology",
        "counseling", "counselling", "pathology", "dermatology", "cardiology",
        "oncology", "pediatric", "geriatric", "therapeutic", "clinical", "dialysis"
    ],
    
    "Essential_services": [
        "bank", "atm", "post office", "utility", "electricity", "water", "gas",
        "postal", "singpost", "supermarket", "grocery", "market", "provision",
        "convenience store", "pharmacy", "petrol", "fuel", "gas station",
        "essential", "basic services", "public utilities", "waste management",
        "recycling", "laundry", "dry cleaning", "repair services", "maintenance",
        "cleaning", "pest control", "plumbing", "electrical", "aircon",
        "air conditioning", "hvac", "installation", "servicing", "telecom",
        "telecommunications", "internet", "broadband", "cable", "satellite",
        "mini mart", "7-eleven", "cheers", "fairprice", "cold storage", "sheng siong"
    ],
    
    "Residential": [
        "residential", "housing", "apartment", "condominium", "condo", "hdb",
        "flat", "estate", "home", "house", "residence", "dwelling", "villa",
        "bungalow", "townhouse", "maisonette", "penthouse", "serviced apartment",
        "dormitory", "hostel", "boarding", "lodging", "quarters", "living",
        "property management", "estate management", "facilities management", "tenant"
    ],
    
    "Education_institutions": [
        "school", "education", "kindergarten", "preschool", "primary", "secondary",
        "university", "college", "polytechnic", "institute", "tuition", "enrichment",
        "training", "learning", "academic", "student", "teacher", "instructor",
        "course", "class", "lesson", "workshop", "seminar", "coaching",
        "educational", "study", "tutorial", "academy", "nursery", "childcare",
        "daycare", "student care", "after school", "language", "music", "art",
        "dance", "martial arts", "swimming coaching", "examination", "skills training"
    ],
    
    "Transport_services": [
        "transport", "mrt", "lrt", "bus", "taxi", "grab", "station", "interchange",
        "terminal", "airport", "changi", "port", "ferry", "boat", "marina",
        "parking", "carpark", "car park", "garage", "vehicle", "automotive",
        "workshop", "service center", "petrol station", "logistics", "courier",
        "delivery", "shipping", "freight", "warehouse", "storage", "moving",
        "relocation", "trucking", "cargo", "forwarding", "express", "postal delivery"
    ],
    
    "Tourism": [
        "hotel", "resort", "hostel", "accommodation", "tourist", "tourism",
        "attraction", "museum", "gallery", "heritage", "cultural", "zoo",
        "aquarium", "theme park", "entertainment", "casino", "cruise",
        "tour", "travel", "vacation", "holiday", "sightseeing", "landmark",
        "monument", "gardens", "park", "beach", "island", "sentosa",
        "marina bay", "orchard", "chinatown", "little india", "adventure",
        "recreation", "amusement", "leisure activities", "excursion", "safari"
    ],
    
    "Community_spaces": [
        "community", "center", "centre", "club", "association", "society",
        "library", "sports", "gym", "fitness", "swimming", "pool", "court",
        "field", "playground", "park", "garden", "recreational", "leisure",
        "activity", "social", "gathering", "meeting", "event", "function",
        "hall", "auditorium", "pavilion", "void deck", "common area",
        "volunteer", "charity", "non-profit", "foundation", "welfare", "ngo"
    ],
    
    "Government_services": [
        "government", "ministry", "statutory board", "public", "civil service",
        "town council", "hdb office", "cpf", "iras", "mom", "moe", "moh",
        "court", "tribunal", "registry", "authority", "agency", "commission",
        "municipal", "grassroots", "pa", "people's association", "cc",
        "community center", "rc", "residents committee", "official",
        "parliament", "embassy", "consulate", "immigration", "customs", "ica"
    ],
    
    "Retail_services": [
        "retail", "shop", "store", "mart", "department store", "shopping",
        "mall", "plaza", "fashion", "clothing", "electronics", "jewelry",
        "jewellery", "hardware", "bookstore", "optical", "sporting goods",
        "furniture", "toys", "pet", "beauty", "cosmetic", "salon", "spa",
        "restaurant", "cafe", "coffee", "bar", "pub", "food court", "dining",
        "catering", "bakery", "food", "beverage", "hawker", "entertainment",
        "cinema", "ktv", "karaoke", "massage", "trading", "wholesale",
        "import", "export", "distribution", "sales", "business", "commercial",
        "manufacturing", "production", "processing", "packaging", "assembly",
        "fabrication", "construction", "building", "renovation", "interior",
        "design", "architecture", "engineering", "consulting", "advisory",
        "professional services", "legal", "accounting", "finance", "insurance",
        "real estate", "property", "investment", "marketing", "advertising",
        "media", "publishing", "printing", "technology", "software", "it",
        "computer", "digital", "online", "internet", "web", "app", "system",
        "development", "programming", "data", "analytics", "research", "supply",
        "services", "company", "enterprise", "corporation", "firm", "office",
        "bureau", "agency", "solutions", "systems", "international", "global"
    ]
}

# ========================================================
# PART 4: Enhanced Classification Function (SAME APPROACH AS ORIGINAL)
# ========================================================

def classify_business_description(description: str, use_mediapipe: bool = True) -> tuple:
    """
    Classify business description using hybrid approach:
    1. MediaPipe for general sentiment/category hints
    2. Keyword matching for specific Singapore business types
    3. ENHANCED fallback logic to minimize 'Others' classification
    """
    
    description_lower = description.lower()
    
    # Method 1: Keyword-based classification (primary) - SAME AS ORIGINAL
    category_scores = {}
    
    for category, keywords in category_keywords.items():
        score = 0
        matched_keywords = []
        
        for keyword in keywords:
            if keyword in description_lower:
                # Weight longer, more specific keywords higher
                weight = len(keyword.split()) * 2 if len(keyword.split()) > 1 else 1
                score += weight
                matched_keywords.append(keyword)
        
        if score > 0:
            category_scores[category] = {
                'score': score,
                'keywords': matched_keywords,
                'confidence': min(score / 10.0, 1.0)  # Normalize confidence
            }
    
    # Method 2: Use MediaPipe for additional insights (SAME AS ORIGINAL)
    mediapipe_result = None
    if use_mediapipe:
        try:
            classification_result = classifier.classify(description)
            if classification_result.classifications:
                # Extract top classification
                top_classification = classification_result.classifications[0]
                if top_classification.categories:
                    top_category = top_classification.categories[0]
                    mediapipe_result = {
                        'category': top_category.category_name,
                        'score': top_category.score
                    }
        except Exception as e:
            print(f"MediaPipe classification error: {e}")
    
    # Method 3: ENHANCED fallback logic for common Singapore business patterns
    if not category_scores:
        # EXPANDED broader keyword matching
        broad_keywords = {
            'service': ['Essential_services', 'Retail_services'],
            'services': ['Essential_services', 'Retail_services'],
            'company': ['Retail_services', 'Essential_services'],
            'center': ['Community_spaces', 'Healthcare_facilities'],
            'centre': ['Community_spaces', 'Healthcare_facilities'],
            'management': ['Retail_services', 'Essential_services'],
            'trading': ['Retail_services'],
            'engineering': ['Essential_services'],
            'development': ['Retail_services'],
            'construction': ['Essential_services'],
            'technology': ['Retail_services'],
            'consulting': ['Retail_services'],
            'solutions': ['Retail_services'],
            'systems': ['Retail_services'],
            'international': ['Retail_services'],
            'industrial': ['Essential_services'],
            'manufacturing': ['Essential_services'],
            # NEW ADDITIONS to reduce Others
            'business': ['Retail_services'],
            'enterprise': ['Retail_services'],
            'corporation': ['Retail_services'],
            'firm': ['Retail_services'],
            'office': ['Retail_services'],
            'agency': ['Retail_services'],
            'bureau': ['Retail_services'],
            'supply': ['Retail_services'],
            'distribution': ['Retail_services'],
            'wholesale': ['Retail_services'],
            'retail': ['Retail_services'],
            'sales': ['Retail_services'],
            'marketing': ['Retail_services'],
            'design': ['Retail_services'],
            'consulting': ['Retail_services'],
            'advisory': ['Retail_services'],
            'professional': ['Retail_services'],
            'legal': ['Retail_services'],
            'financial': ['Retail_services'],
            'investment': ['Retail_services'],
            'insurance': ['Retail_services'],
            'property': ['Retail_services'],
            'maintenance': ['Essential_services'],
            'repair': ['Essential_services'],
            'installation': ['Essential_services'],
            'cleaning': ['Essential_services'],
            'waste': ['Essential_services'],
            'utility': ['Essential_services'],
            'transport': ['Transport_services'],
            'logistics': ['Transport_services'],
            'delivery': ['Transport_services'],
            'shipping': ['Transport_services'],
            'freight': ['Transport_services'],
            'storage': ['Transport_services'],
            'warehouse': ['Transport_services']
        }
        
        for keyword, potential_categories in broad_keywords.items():
            if keyword in description_lower:
                # Assign to most likely category with lower confidence
                category_scores[potential_categories[0]] = {
                    'score': 1,
                    'keywords': [keyword],
                    'confidence': 0.4
                }
                break
        
        # FINAL FALLBACK: If still no match, try even more general patterns
        if not category_scores:
            # Look for common business structure words
            business_indicators = ['pte', 'ltd', 'llp', 'limited', 'private', 'public', 'holdings', 'group']
            if any(indicator in description_lower for indicator in business_indicators):
                category_scores['Retail_services'] = {
                    'score': 1,
                    'keywords': ['business_entity'],
                    'confidence': 0.3
                }
            # Look for action words that suggest commercial activity
            elif any(word in description_lower for word in ['sale', 'selling', 'purchase', 'buy', 'sell', 'trade', 'deal']):
                category_scores['Retail_services'] = {
                    'score': 1,
                    'keywords': ['commercial_activity'],
                    'confidence': 0.3
                }
            # Default for multi-word descriptions (likely business)
            elif len(description_lower.split()) >= 3:
                category_scores['Retail_services'] = {
                    'score': 1,
                    'keywords': ['default_business'],
                    'confidence': 0.2
                }
    
    # Final decision logic (SAME AS ORIGINAL)
    if category_scores:
        # Get category with highest score
        best_category = max(category_scores.items(), key=lambda x: x[1]['score'])
        
        primary_category = best_category[0]
        confidence = best_category[1]['confidence']
        keywords_matched = best_category[1]['keywords']
        
        # Get secondary categories if they exist
        secondary_categories = [cat for cat, data in category_scores.items() 
                             if cat != primary_category and data['score'] >= 2]
        
        all_categories = [primary_category] + secondary_categories[:2]  # Max 3 categories
        
        return all_categories, confidence, keywords_matched
    
    else:
        # Last resort: default to Others (should be much less now)
        return ['Others'], 0.2, ['no_keywords_matched']

# ========================================================
# PART 5: Apply Classification with Resume Capability
# ========================================================

print("Starting enhanced classification with new categories...")

# Batch processing configuration
BATCH_SIZE = 10000
total_records = len(df)
num_batches = (total_records + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division
main_output_path = "acra_classified_full_dataset.csv"

print(f"Total dataset: {total_records:,} records in {num_batches} batches of {BATCH_SIZE:,} records each...")

# Check completion status
last_completed_batch, records_processed, should_resume = check_batch_completion(main_output_path, BATCH_SIZE)

if should_resume and last_completed_batch >= 31:  # Batch 32 or later
    print(f"\n🔄 RESUMING FROM INTERRUPTION")
    print(f"Last completed batch: {last_completed_batch}")
    
    # Determine output files for remaining batches
    remaining_batches = num_batches - last_completed_batch
    
    if remaining_batches <= 30:
        # All remaining batches go to one file
        output_files = [("acra_classified_part2.csv", remaining_batches)]
        print(f"📁 Remaining {remaining_batches} batches will be saved to: acra_classified_part2.csv")
    else:
        # Split remaining batches into two files
        first_part_batches = 30
        second_part_batches = remaining_batches - 30
        output_files = [
            ("acra_classified_part2.csv", first_part_batches),
            ("acra_classified_part3.csv", second_part_batches)
        ]
        print(f"📁 Next 30 batches will be saved to: acra_classified_part2.csv")
        print(f"📁 Final {second_part_batches} batches will be saved to: acra_classified_part3.csv")
    
    start_batch = last_completed_batch
    
else:
    # Start from beginning or continue normal processing
    if should_resume:
        print(f"✅ Continuing from batch {last_completed_batch + 1}")
        start_batch = last_completed_batch
    else:
        print("🆕 Starting fresh processing")
        start_batch = 0
    
    # For normal processing, use original file
    output_files = [(main_output_path, num_batches)]

# Process in batches with multiple output files
current_file_index = 0
batches_in_current_file = 0
current_output_path = output_files[current_file_index][0]
max_batches_current_file = output_files[current_file_index][1]

print(f"\n🚀 Starting batch processing from batch {start_batch + 1}...")

for batch_num in range(start_batch, num_batches):
    # Check if we need to switch to next output file
    if batches_in_current_file >= max_batches_current_file and current_file_index < len(output_files) - 1:
        current_file_index += 1
        current_output_path = output_files[current_file_index][0]
        max_batches_current_file = output_files[current_file_index][1]
        batches_in_current_file = 0
        print(f"\n📁 Switching to new output file: {current_output_path}")
    
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, total_records)
    
    print(f"\n--- Processing Batch {batch_num + 1}/{num_batches} ---")
    print(f"Records {start_idx:,} to {end_idx-1:,}")
    print(f"Output file: {current_output_path}")
    
    # Get current batch
    df_batch = df.iloc[start_idx:end_idx].copy()
    
    # Process batch (SAME AS ORIGINAL)
    results = []
    for desc in tqdm(df_batch[col_name].astype(str).tolist(), 
                     desc=f"Batch {batch_num + 1}/{num_batches}"):
        result = classify_business_description(desc)
        results.append(result)
    
    # Unpack results (SAME AS ORIGINAL)
    categories_list, confidences, keywords_matched = zip(*results)
    
    df_batch['amenity_categories'] = [', '.join(cats) for cats in categories_list]
    df_batch['classification_confidence'] = confidences
    df_batch['keywords_matched'] = [', '.join(kw) for kw in keywords_matched]
    
    # Save batch results
    if batches_in_current_file == 0:
        # First batch in this file - create new file with headers
        df_batch.to_csv(current_output_path, index=False, mode='w')
        print(f"Created new file: {current_output_path}")
    else:
        # Subsequent batches - append without headers
        df_batch.to_csv(current_output_path, index=False, mode='a', header=False)
        print(f"Appended batch {batch_num + 1} to {current_output_path}")
    
    batches_in_current_file += 1
    
    # Quick batch analysis (SAME AS ORIGINAL)
    batch_category_counts = (
        df_batch['amenity_categories']
        .str.split(', ')
        .explode()
        .value_counts()
    )
    
    batch_others_pct = (batch_category_counts.get('Others', 0) / len(df_batch)) * 100
    print(f"Batch {batch_num + 1} 'Others' percentage: {batch_others_pct:.1f}%")
    
    # Memory cleanup (SAME AS ORIGINAL)
    del df_batch, results, categories_list, confidences, keywords_matched
    import gc
    gc.collect()

print(f"\n🎉 BATCH PROCESSING COMPLETE!")

# Display summary of output files
print(f"\n📊 OUTPUT FILES SUMMARY:")
for file_path, expected_batches in output_files:
    if os.path.exists(file_path):
        try:
            df_check = pd.read_csv(file_path)
            actual_records = len(df_check)
            actual_batches = (actual_records + BATCH_SIZE - 1) // BATCH_SIZE
            print(f"📁 {file_path}: {actual_records:,} records ({actual_batches} batches)")
        except Exception as e:
            print(f"📁 {file_path}: Error reading file - {e}")
    else:
        print(f"📁 {file_path}: File not created")

# ========================================================
# PART 6: Final Analysis (Read back results from all files)
# ========================================================

print(f"\n📈 COMBINING RESULTS FROM ALL FILES...")

# Combine all output files for analysis
all_dataframes = []
total_processed = 0

for file_path, _ in output_files:
    if os.path.exists(file_path):
        try:
            df_part = pd.read_csv(file_path)
            all_dataframes.append(df_part)
            total_processed += len(df_part)
            print(f"✅ Loaded {len(df_part):,} records from {file_path}")
        except Exception as e:
            print(f"❌ Error loading {file_path}: {e}")

if all_dataframes:
    # Combine all dataframes
    df_final = pd.concat(all_dataframes, ignore_index=True)
    print(f"\n🔢 Combined dataset: {len(df_final):,} total records")
    
    # Save combined results
    combined_output_path = "acra_classified_combined_results.csv"
    df_final.to_csv(combined_output_path, index=False)
    print(f"💾 Combined results saved to: {combined_output_path}")
    
    # ========================================================
    # PART 7: Detailed Analysis (SAME AS ORIGINAL)
    # ========================================================
    
    print("\n" + "="*60)
    print("FINAL CLASSIFICATION RESULTS - COMBINED DATASET")
    print("="*60)
    
    # Category distribution
    category_counts = (
        df_final['amenity_categories']
        .str.split(', ')
        .explode()
        .value_counts()
    )
    
    print(f"\nCategory Distribution:")
    for category, count in category_counts.items():
        percentage = (count / len(df_final)) * 100
        print(f"{category:25}: {count:,} ({percentage:5.1f}%)")
    
    # Key improvements
    others_pct = (category_counts.get('Others', 0) / len(df_final)) * 100
    print(f"\n🎯 'Others' percentage: {others_pct:.1f}%")
    
    # Confidence analysis
    high_conf = len(df_final[df_final['classification_confidence'] >= 0.7])
    medium_conf = len(df_final[(df_final['classification_confidence'] >= 0.4) & 
                               (df_final['classification_confidence'] < 0.7)])
    low_conf = len(df_final[df_final['classification_confidence'] < 0.4])
    
    print(f"\nConfidence Distribution:")
    print(f"High confidence (≥0.7): {high_conf:,} ({high_conf/len(df_final)*100:.1f}%)")
    print(f"Medium confidence (0.4-0.7): {medium_conf:,} ({medium_conf/len(df_final)*100:.1f}%)")
    print(f"Low confidence (<0.4): {low_conf:,} ({low_conf/len(df_final)*100:.1f}%)")
    
    # Sample results
    print(f"\nSample Results (first 15 records):")
    print("-" * 120)
    for idx in range(min(15, len(df_final))):
        row = df_final.iloc[idx]
        desc = row[col_name][:50] + "..." if len(row[col_name]) > 50 else row[col_name]
        print(f"Business: {desc:52} → {row['amenity_categories']:25} "
              f"({row['classification_confidence']:.2f}) [{row['keywords_matched'][:30]}]")
    
    print(f"\n✅ SUCCESS! Classification completed!")
    print(f"📊 Total processed: {len(df_final):,} records")
    
    # Final summary
    print(f"\n🔮 FINAL RESULTS SUMMARY:")
    print(f"Total records processed: {len(df_final):,}")
    print(f"'Others' category: {others_pct:.1f}% ({category_counts.get('Others', 0):,} records)")
    print(f"Successfully classified: {100 - others_pct:.1f}%")
    
    # Show file breakdown
    print(f"\n📁 OUTPUT FILES:")
    for file_path, _ in output_files:
        if os.path.exists(file_path):
            try:
                df_check = pd.read_csv(file_path)
                print(f"   {file_path}: {len(df_check):,} records")
            except:
                print(f"   {file_path}: Error reading")
    
    print(f"   {combined_output_path}: {len(df_final):,} records (combined)")

else:
    print("❌ No data available for final analysis")

print(f"\n🏁 PROCESSING COMPLETE!")
print(f"Check the output files for your classified data.")

Setting up MediaPipe Text Classifier...


I0000 00:00:1757151264.463498   15117 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1757151264.677622   42381 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Loading ACRA data...
Dataset Shape: (951620, 15)
Starting enhanced classification with new categories...
Total dataset: 951,620 records in 96 batches of 10,000 records each...
Found existing file: acra_classified_full_dataset.csv
Records already processed: 370,000
Complete batches: 37
✅ All batches up to batch 37 are complete

🔄 RESUMING FROM INTERRUPTION
Last completed batch: 37
📁 Next 30 batches will be saved to: acra_classified_part2.csv
📁 Final 29 batches will be saved to: acra_classified_part3.csv

🚀 Starting batch processing from batch 38...

--- Processing Batch 38/96 ---
Records 370,000 to 379,999
Output file: acra_classified_part2.csv


Batch 38/96: 100%|██████████| 10000/10000 [04:22<00:00, 38.02it/s]


Created new file: acra_classified_part2.csv
Batch 38 'Others' percentage: 0.2%

--- Processing Batch 39/96 ---
Records 380,000 to 389,999
Output file: acra_classified_part2.csv


...

Appended batch 94 to acra_classified_part3.csv
Batch 94 'Others' percentage: 0.3%

--- Processing Batch 95/96 ---
Records 940,000 to 949,999
Output file: acra_classified_part3.csv


Batch 95/96: 100%|██████████| 10000/10000 [04:20<00:00, 38.34it/s]


Appended batch 95 to acra_classified_part3.csv
Batch 95 'Others' percentage: 0.4%

--- Processing Batch 96/96 ---
Records 950,000 to 951,619
Output file: acra_classified_part3.csv


Batch 96/96: 100%|██████████| 1620/1620 [00:42<00:00, 37.88it/s]


Appended batch 96 to acra_classified_part3.csv
Batch 96 'Others' percentage: 0.4%

🎉 BATCH PROCESSING COMPLETE!

📊 OUTPUT FILES SUMMARY:
📁 acra_classified_part2.csv: 300,000 records (30 batches)
📁 acra_classified_part3.csv: 281,620 records (29 batches)

📈 COMBINING RESULTS FROM ALL FILES...
✅ Loaded 300,000 records from acra_classified_part2.csv
✅ Loaded 281,620 records from acra_classified_part3.csv

🔢 Combined dataset: 581,620 total records
💾 Combined results saved to: acra_classified_combined_results.csv

FINAL CLASSIFICATION RESULTS - COMBINED DATASET

Category Distribution:
Retail_services          : 416,378 ( 71.6%)
Government_services      : 72,945 ( 12.5%)
Essential_services       : 48,010 (  8.3%)
Transport_services       : 29,681 (  5.1%)
Education_institutions   : 26,931 (  4.6%)
Healthcare_facilities    : 15,712 (  2.7%)
Community_spaces         : 8,289 (  1.4%)
Tourism                  : 5,680 (  1.0%)
Residential              : 4,381 (  0.8%)
Others                   : 2,

In [5]:
# =====================================================================
# ACRA Amenity Classification with MediaPipe Text Classifier
# UPDATED: Removed Method 3 fallback logic to reduce Retail_services bias
# Testing on 5000 records sample
# =====================================================================

import pandas as pd
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text
import numpy as np
from tqdm import tqdm
import urllib.request
import os

# ========================================================
# PART 1: Setup MediaPipe Text Classifier
# ========================================================

print("Setting up MediaPipe Text Classifier...")

# Download the pre-trained model (this is free and runs locally)
model_url = "https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite"
model_path = "bert_classifier.tflite"

if not os.path.exists(model_path):
    print("Downloading MediaPipe text classification model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded successfully!")

# Create the text classifier
base_options = python.BaseOptions(model_asset_path=model_path)
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# ========================================================
# PART 2: Load Dataset (Sample of 5000 records)
# ========================================================

print("Loading ACRA data...")
df_full = pd.read_csv("acra_merged_active.csv")
print("Full Dataset Shape:", df_full.shape)

# Take a sample of 5000 records for testing
df = df_full.sample(n=5000, random_state=42).reset_index(drop=True)
print("Sample Dataset Shape:", df.shape)

if "primary_ssic_description" in df.columns:
    col_name = "primary_ssic_description"
elif "primary_ssic" in df.columns:
    col_name = "primary_ssic"
else:
    raise ValueError("No SSIC description column found in CSV")

# Drop rows with empty descriptions
df = df.dropna(subset=[col_name])
df[col_name] = df[col_name].astype(str).str.strip()

print(f"Final sample size after cleaning: {len(df):,} records")

# ========================================================
# PART 3: Classification Logic with Keywords (Same as original)
# ========================================================

# Define comprehensive keyword mappings for Singapore context
category_keywords = {
    "Emergency_services": [
        "emergency", "ambulance", "fire", "police", "rescue", "paramedic",
        "emergency medical", "fire station", "police station", "civil defence",
        "scdf", "spf", "disaster", "crisis", "urgent care", "first aid",
        "emergency response", "safety", "security services", "emergency care",
        "security", "guard", "surveillance", "protection", "patrol", "investigation"
    ],
    
    "Healthcare_facilities": [
        "hospital", "clinic", "medical", "health", "dental", "doctor", "physician",
        "specialist", "diagnostic", "laboratory", "physiotherapy", "tcm",
        "traditional chinese medicine", "veterinary", "vet", "mental health",
        "rehabilitation", "nursing", "healthcare", "medicine", "pharmaceutical",
        "wellness", "therapy", "treatment", "surgery", "radiology", "dentist",
        "medical center", "health screening", "pharmacy", "medical practice",
        "acupuncture", "chiropractic", "optometry", "podiatry", "psychology",
        "counseling", "counselling", "pathology", "dermatology", "cardiology",
        "oncology", "pediatric", "geriatric", "therapeutic", "clinical", "dialysis"
    ],
    
    "Essential_services": [
        "bank", "atm", "post office", "utility", "electricity", "water", "gas",
        "postal", "singpost", "supermarket", "grocery", "market", "provision",
        "convenience store", "pharmacy", "petrol", "fuel", "gas station",
        "essential", "basic services", "public utilities", "waste management",
        "recycling", "laundry", "dry cleaning", "repair services", "maintenance",
        "cleaning", "pest control", "plumbing", "electrical", "aircon",
        "air conditioning", "hvac", "installation", "servicing", "telecom",
        "telecommunications", "internet", "broadband", "cable", "satellite",
        "mini mart", "7-eleven", "cheers", "fairprice", "cold storage", "sheng siong"
    ],
    
    "Residential": [
        "residential", "housing", "apartment", "condominium", "condo", "hdb",
        "flat", "estate", "home", "house", "residence", "dwelling", "villa",
        "bungalow", "townhouse", "maisonette", "penthouse", "serviced apartment",
        "dormitory", "hostel", "boarding", "lodging", "quarters", "living",
        "property management", "estate management", "facilities management", "tenant"
    ],
    
    "Education_institutions": [
        "school", "education", "kindergarten", "preschool", "primary", "secondary",
        "university", "college", "polytechnic", "institute", "tuition", "enrichment",
        "training", "learning", "academic", "student", "teacher", "instructor",
        "course", "class", "lesson", "workshop", "seminar", "coaching",
        "educational", "study", "tutorial", "academy", "nursery", "childcare",
        "daycare", "student care", "after school", "language", "music", "art",
        "dance", "martial arts", "swimming coaching", "examination", "skills training"
    ],
    
    "Transport_services": [
        "transport", "mrt", "lrt", "bus", "taxi", "grab", "station", "interchange",
        "terminal", "airport", "changi", "port", "ferry", "boat", "marina",
        "parking", "carpark", "car park", "garage", "vehicle", "automotive",
        "workshop", "service center", "petrol station", "logistics", "courier",
        "delivery", "shipping", "freight", "warehouse", "storage", "moving",
        "relocation", "trucking", "cargo", "forwarding", "express", "postal delivery"
    ],
    
    "Tourism": [
        "hotel", "resort", "hostel", "accommodation", "tourist", "tourism",
        "attraction", "museum", "gallery", "heritage", "cultural", "zoo",
        "aquarium", "theme park", "entertainment", "casino", "cruise",
        "tour", "travel", "vacation", "holiday", "sightseeing", "landmark",
        "monument", "gardens", "park", "beach", "island", "sentosa",
        "marina bay", "orchard", "chinatown", "little india", "adventure",
        "recreation", "amusement", "leisure activities", "excursion", "safari"
    ],
    
    "Community_spaces": [
        "community", "center", "centre", "club", "association", "society",
        "library", "sports", "gym", "fitness", "swimming", "pool", "court",
        "field", "playground", "park", "garden", "recreational", "leisure",
        "activity", "social", "gathering", "meeting", "event", "function",
        "hall", "auditorium", "pavilion", "void deck", "common area",
        "volunteer", "charity", "non-profit", "foundation", "welfare", "ngo"
    ],
    
    "Government_services": [
        "government", "ministry", "statutory board", "public", "civil service",
        "town council", "hdb office", "cpf", "iras", "mom", "moe", "moh",
        "court", "tribunal", "registry", "authority", "agency", "commission",
        "municipal", "grassroots", "pa", "people's association", "cc",
        "community center", "rc", "residents committee", "official",
        "parliament", "embassy", "consulate", "immigration", "customs", "ica"
    ],
    
    "Retail_services": [
        "retail", "shop", "store", "mart", "department store", "shopping",
        "mall", "plaza", "fashion", "clothing", "electronics", "jewelry",
        "jewellery", "hardware", "bookstore", "optical", "sporting goods",
        "furniture", "toys", "pet", "beauty", "cosmetic", "salon", "spa",
        "restaurant", "cafe", "coffee", "bar", "pub", "food court", "dining",
        "catering", "bakery", "food", "beverage", "hawker", "entertainment",
        "cinema", "ktv", "karaoke", "massage", "trading", "wholesale",
        "import", "export", "distribution", "sales", "business", "commercial",
        "manufacturing", "production", "processing", "packaging", "assembly",
        "fabrication", "construction", "building", "renovation", "interior",
        "design", "architecture", "engineering", "consulting", "advisory",
        "professional services", "legal", "accounting", "finance", "insurance",
        "real estate", "property", "investment", "marketing", "advertising",
        "media", "publishing", "printing", "technology", "software", "it",
        "computer", "digital", "online", "internet", "web", "app", "system",
        "development", "programming", "data", "analytics", "research"
    ]
}

# ========================================================
# PART 4: UPDATED Classification Function (REMOVED METHOD 3)
# ========================================================

def classify_business_description_no_method3(description: str, use_mediapipe: bool = True) -> tuple:
    """
    Classify business description using ONLY:
    1. MediaPipe for general sentiment/category hints
    2. Keyword matching for specific Singapore business types
    3. NO aggressive fallback logic (removed Method 3)
    """
    
    description_lower = description.lower()
    
    # Method 1: Keyword-based classification (primary)
    category_scores = {}
    
    for category, keywords in category_keywords.items():
        score = 0
        matched_keywords = []
        
        for keyword in keywords:
            if keyword in description_lower:
                # Weight longer, more specific keywords higher
                weight = len(keyword.split()) * 2 if len(keyword.split()) > 1 else 1
                score += weight
                matched_keywords.append(keyword)
        
        if score > 0:
            category_scores[category] = {
                'score': score,
                'keywords': matched_keywords,
                'confidence': min(score / 10.0, 1.0)  # Normalize confidence
            }
    
    # Method 2: Use MediaPipe for additional insights
    mediapipe_result = None
    if use_mediapipe:
        try:
            classification_result = classifier.classify(description)
            if classification_result.classifications:
                # Extract top classification
                top_classification = classification_result.classifications[0]
                if top_classification.categories:
                    top_category = top_classification.categories[0]
                    mediapipe_result = {
                        'category': top_category.category_name,
                        'score': top_category.score
                    }
        except Exception as e:
            print(f"MediaPipe classification error: {e}")
    
    # REMOVED METHOD 3 - No aggressive fallback logic
    
    # Final decision logic
    if category_scores:
        # Get category with highest score
        best_category = max(category_scores.items(), key=lambda x: x[1]['score'])
        
        primary_category = best_category[0]
        confidence = best_category[1]['confidence']
        keywords_matched = best_category[1]['keywords']
        
        # Get secondary categories if they exist
        secondary_categories = [cat for cat, data in category_scores.items() 
                             if cat != primary_category and data['score'] >= 2]
        
        all_categories = [primary_category] + secondary_categories[:2]  # Max 3 categories
        
        return all_categories, confidence, keywords_matched
    
    else:
        # Only assign to Others if truly no keywords matched
        return ['Others'], 0.1, ['no_keywords_matched']

# ========================================================
# PART 5: Apply Classification to Sample
# ========================================================

print("Starting classification without Method 3 fallback logic...")
print(f"Processing {len(df):,} records...")

# Process sample
results = []
for desc in tqdm(df[col_name].astype(str).tolist(), desc="Classifying"):
    result = classify_business_description_no_method3(desc)
    results.append(result)

# Unpack results
categories_list, confidences, keywords_matched = zip(*results)

df['amenity_categories'] = [', '.join(cats) for cats in categories_list]
df['classification_confidence'] = confidences
df['keywords_matched'] = [', '.join(kw) for kw in keywords_matched]

# Save results
output_file = "acra_sample_no_method3.csv"
df.to_csv(output_file, index=False)
print(f"Sample results saved to: {output_file}")

# ========================================================
# PART 6: Analysis and Comparison
# ========================================================

print("\n" + "="*60)
print("CLASSIFICATION RESULTS - WITHOUT METHOD 3")
print("="*60)

# Category distribution
category_counts = (
    df['amenity_categories']
    .str.split(', ')
    .explode()
    .value_counts()
)

print(f"\nCategory Distribution:")
for category, count in category_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{category:25}: {count:,} ({percentage:5.1f}%)")

# Key metrics
retail_pct = (category_counts.get('Retail_services', 0) / len(df)) * 100
others_pct = (category_counts.get('Others', 0) / len(df)) * 100

print(f"\n🎯 Key Metrics:")
print(f"Retail_services: {retail_pct:.1f}%")
print(f"Others: {others_pct:.1f}%")

# Confidence analysis
high_conf = len(df[df['classification_confidence'] >= 0.7])
medium_conf = len(df[(df['classification_confidence'] >= 0.4) & 
                     (df['classification_confidence'] < 0.7)])
low_conf = len(df[df['classification_confidence'] < 0.4])

print(f"\nConfidence Distribution:")
print(f"High confidence (≥0.7): {high_conf:,} ({high_conf/len(df)*100:.1f}%)")
print(f"Medium confidence (0.4-0.7): {medium_conf:,} ({medium_conf/len(df)*100:.1f}%)")
print(f"Low confidence (<0.4): {low_conf:,} ({low_conf/len(df)*100:.1f}%)")

# Show examples of each category
print(f"\n📋 SAMPLE CLASSIFICATIONS BY CATEGORY:")
print("-" * 100)

for category in category_counts.index[:10]:  # Top 10 categories
    sample_records = df[df['amenity_categories'].str.contains(category, na=False)].head(3)
    print(f"\n{category}:")
    for idx, row in sample_records.iterrows():
        desc = row[col_name][:60] + "..." if len(row[col_name]) > 60 else row[col_name]
        print(f"  • {desc} (conf: {row['classification_confidence']:.2f})")

# Show Others category examples for analysis
others_examples = df[df['amenity_categories'] == 'Others'].head(10)
print(f"\n❓ EXAMPLES OF 'Others' CATEGORY:")
print("-" * 100)
for idx, row in others_examples.iterrows():
    desc = row[col_name][:80] + "..." if len(row[col_name]) > 80 else row[col_name]
    print(f"  • {desc}")

print(f"\n✅ ANALYSIS COMPLETE!")
print(f"📊 Total sample processed: {len(df):,} records")
print(f"📉 Retail_services reduced from ~70% to {retail_pct:.1f}%")
print(f"📈 Others increased to {others_pct:.1f}%")
print(f"💾 Results saved to: {output_file}")

print(f"\n🔍 RECOMMENDATION:")
if retail_pct < 30:
    print("✅ Retail_services percentage looks much more reasonable now!")
    print("✅ The removal of Method 3 has successfully reduced the bias.")
    print("💡 You can now apply this updated logic to your full dataset.")
else:
    print("⚠️  Retail_services is still high. Consider refining the keywords further.")

if others_pct > 40:
    print("📝 High 'Others' percentage suggests we might need some targeted keywords")
    print("📝 Review the 'Others' examples above to identify common patterns")
else:
    print("✅ 'Others' percentage is reasonable.")

Setting up MediaPipe Text Classifier...


I0000 00:00:1757170567.708837   15117 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1757170567.922912  335854 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Loading ACRA data...
Full Dataset Shape: (951620, 15)
Sample Dataset Shape: (5000, 15)
Final sample size after cleaning: 5,000 records
Starting classification without Method 3 fallback logic...
Processing 5,000 records...


Classifying: 100%|██████████| 5000/5000 [02:15<00:00, 37.03it/s]


Sample results saved to: acra_sample_no_method3.csv

CLASSIFICATION RESULTS - WITHOUT METHOD 3

Category Distribution:
Retail_services          : 2,919 ( 58.4%)
Government_services      : 685 ( 13.7%)
Others                   : 475 (  9.5%)
Essential_services       : 398 (  8.0%)
Transport_services       : 295 (  5.9%)
Education_institutions   : 264 (  5.3%)
Healthcare_facilities    : 150 (  3.0%)
Community_spaces         : 72 (  1.4%)
Tourism                  : 62 (  1.2%)
Residential              : 41 (  0.8%)
Emergency_services       : 24 (  0.5%)

🎯 Key Metrics:
Retail_services: 58.4%
Others: 9.5%

Confidence Distribution:
High confidence (≥0.7): 60 (1.2%)
Medium confidence (0.4-0.7): 433 (8.7%)
Low confidence (<0.4): 4,507 (90.1%)

📋 SAMPLE CLASSIFICATIONS BY CATEGORY:
----------------------------------------------------------------------------------------------------

Retail_services:
  • RETAIL SALE OF HOUSEHOLD ELECTRICAL APPLIANCES AND EQUIPMENT... (conf: 0.20)
  • DEPARTMENT 

In [ ]:
# =====================================================================
# ACRA Amenity Classification - FULL DATASET PROCESSING
# Process entire dataset in batches of 10,000 records
# Split across 3 output files: acra_classified_part1v2.csv, part2v2.csv, part3v2.csv
# Uses refined classification without Method 3 fallback logic
# =====================================================================

import pandas as pd
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text
import numpy as np
from tqdm import tqdm
import urllib.request
import os
import gc

# ========================================================
# PART 0: Configuration
# ========================================================

BATCH_SIZE = 10000
OUTPUT_FILES = [
    "acra_classified_part1v2.csv",
    "acra_classified_part2v2.csv", 
    "acra_classified_part3v2.csv"
]

print("="*60)
print("ACRA FULL DATASET CLASSIFICATION - BATCH PROCESSING")
print("="*60)
print(f"Batch size: {BATCH_SIZE:,} records")
print(f"Output files: {OUTPUT_FILES}")
print("="*60)

# ========================================================
# PART 1: Setup MediaPipe Text Classifier
# ========================================================

print("Setting up MediaPipe Text Classifier...")

model_url = "https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite"
model_path = "bert_classifier.tflite"

if not os.path.exists(model_path):
    print("Downloading MediaPipe text classification model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Model downloaded successfully!")

base_options = python.BaseOptions(model_asset_path=model_path)
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# ========================================================
# PART 2: Load Full Dataset
# ========================================================

print("Loading full ACRA dataset...")
df = pd.read_csv("acra_merged_active.csv")
print(f"Full Dataset Shape: {df.shape}")

if "primary_ssic_description" in df.columns:
    col_name = "primary_ssic_description"
elif "primary_ssic" in df.columns:
    col_name = "primary_ssic"
else:
    raise ValueError("No SSIC description column found in CSV")

# Clean dataset
df = df.dropna(subset=[col_name])
df[col_name] = df[col_name].astype(str).str.strip()

total_records = len(df)
total_batches = (total_records + BATCH_SIZE - 1) // BATCH_SIZE

print(f"Final dataset size: {total_records:,} records")
print(f"Total batches: {total_batches}")

# Calculate batches per file
batches_per_file = total_batches // 3
extra_batches = total_batches % 3

file_batch_counts = [batches_per_file] * 3
for i in range(extra_batches):
    file_batch_counts[i] += 1

print(f"Batches per file: {file_batch_counts}")
print(f"Records per file: {[count * BATCH_SIZE for count in file_batch_counts]}")

# ========================================================
# PART 3: Classification Keywords (No Method 3)
# ========================================================

category_keywords = {
    "Emergency_services": [
        "emergency", "ambulance", "fire", "police", "rescue", "paramedic",
        "emergency medical", "fire station", "police station", "civil defence",
        "scdf", "spf", "disaster", "crisis", "urgent care", "first aid",
        "emergency response", "safety", "security services", "emergency care",
        "security", "guard", "surveillance", "protection", "patrol", "investigation"
    ],
    
    "Healthcare_facilities": [
        "hospital", "clinic", "medical", "health", "dental", "doctor", "physician",
        "specialist", "diagnostic", "laboratory", "physiotherapy", "tcm",
        "traditional chinese medicine", "veterinary", "vet", "mental health",
        "rehabilitation", "nursing", "healthcare", "medicine", "pharmaceutical",
        "wellness", "therapy", "treatment", "surgery", "radiology", "dentist",
        "medical center", "health screening", "pharmacy", "medical practice",
        "acupuncture", "chiropractic", "optometry", "podiatry", "psychology",
        "counseling", "counselling", "pathology", "dermatology", "cardiology",
        "oncology", "pediatric", "geriatric", "therapeutic", "clinical", "dialysis"
    ],
    
    "Essential_services": [
        "bank", "atm", "post office", "utility", "electricity", "water", "gas",
        "postal", "singpost", "supermarket", "grocery", "market", "provision",
        "convenience store", "pharmacy", "petrol", "fuel", "gas station",
        "essential", "basic services", "public utilities", "waste management",
        "recycling", "laundry", "dry cleaning", "repair services", "maintenance",
        "cleaning", "pest control", "plumbing", "electrical", "aircon",
        "air conditioning", "hvac", "installation", "servicing", "telecom",
        "telecommunications", "internet", "broadband", "cable", "satellite",
        "mini mart", "7-eleven", "cheers", "fairprice", "cold storage", "sheng siong"
    ],
    
    "Residential": [
        "residential", "housing", "apartment", "condominium", "condo", "hdb",
        "flat", "estate", "home", "house", "residence", "dwelling", "villa",
        "bungalow", "townhouse", "maisonette", "penthouse", "serviced apartment",
        "dormitory", "hostel", "boarding", "lodging", "quarters", "living",
        "property management", "estate management", "facilities management", "tenant"
    ],
    
    "Education_institutions": [
        "school", "education", "kindergarten", "preschool", "primary", "secondary",
        "university", "college", "polytechnic", "institute", "tuition", "enrichment",
        "training", "learning", "academic", "student", "teacher", "instructor",
        "course", "class", "lesson", "workshop", "seminar", "coaching",
        "educational", "study", "tutorial", "academy", "nursery", "childcare",
        "daycare", "student care", "after school", "language", "music", "art",
        "dance", "martial arts", "swimming coaching", "examination", "skills training"
    ],
    
    "Transport_services": [
        "transport", "mrt", "lrt", "bus", "taxi", "grab", "station", "interchange",
        "terminal", "airport", "changi", "port", "ferry", "boat", "marina",
        "parking", "carpark", "car park", "garage", "vehicle", "automotive",
        "workshop", "service center", "petrol station", "logistics", "courier",
        "delivery", "shipping", "freight", "warehouse", "storage", "moving",
        "relocation", "trucking", "cargo", "forwarding", "express", "postal delivery"
    ],
    
    "Tourism": [
        "hotel", "resort", "hostel", "accommodation", "tourist", "tourism",
        "attraction", "museum", "gallery", "heritage", "cultural", "zoo",
        "aquarium", "theme park", "entertainment", "casino", "cruise",
        "tour", "travel", "vacation", "holiday", "sightseeing", "landmark",
        "monument", "gardens", "park", "beach", "island", "sentosa",
        "marina bay", "orchard", "chinatown", "little india", "adventure",
        "recreation", "amusement", "leisure activities", "excursion", "safari"
    ],
    
    "Community_spaces": [
        "community", "center", "centre", "club", "association", "society",
        "library", "sports", "gym", "fitness", "swimming", "pool", "court",
        "field", "playground", "park", "garden", "recreational", "leisure",
        "activity", "social", "gathering", "meeting", "event", "function",
        "hall", "auditorium", "pavilion", "void deck", "common area",
        "volunteer", "charity", "non-profit", "foundation", "welfare", "ngo"
    ],
    
    "Government_services": [
        "government", "ministry", "statutory board", "public", "civil service",
        "town council", "hdb office", "cpf", "iras", "mom", "moe", "moh",
        "court", "tribunal", "registry", "authority", "agency", "commission",
        "municipal", "grassroots", "pa", "people's association", "cc",
        "community center", "rc", "residents committee", "official",
        "parliament", "embassy", "consulate", "immigration", "customs", "ica"
    ],
    
    "Retail_services": [
        "retail", "shop", "store", "mart", "department store", "shopping",
        "mall", "plaza", "fashion", "clothing", "electronics", "jewelry",
        "jewellery", "hardware", "bookstore", "optical", "sporting goods",
        "furniture", "toys", "pet", "beauty", "cosmetic", "salon", "spa",
        "restaurant", "cafe", "coffee", "bar", "pub", "food court", "dining",
        "catering", "bakery", "food", "beverage", "hawker", "entertainment",
        "cinema", "ktv", "karaoke", "massage", "trading", "wholesale",
        "import", "export", "distribution", "sales", "business", "commercial",
        "manufacturing", "production", "processing", "packaging", "assembly",
        "fabrication", "construction", "building", "renovation", "interior",
        "design", "architecture", "engineering", "consulting", "advisory",
        "professional services", "legal", "accounting", "finance", "insurance",
        "real estate", "property", "investment", "marketing", "advertising",
        "media", "publishing", "printing", "technology", "software", "it",
        "computer", "digital", "online", "internet", "web", "app", "system",
        "development", "programming", "data", "analytics", "research"
    ]
}

# ========================================================
# PART 4: Classification Function (No Method 3)
# ========================================================

def classify_business_description(description: str, use_mediapipe: bool = True) -> tuple:
    """
    Classify business description without Method 3 fallback logic
    """
    description_lower = description.lower()
    
    # Method 1: Keyword-based classification
    category_scores = {}
    
    for category, keywords in category_keywords.items():
        score = 0
        matched_keywords = []
        
        for keyword in keywords:
            if keyword in description_lower:
                weight = len(keyword.split()) * 2 if len(keyword.split()) > 1 else 1
                score += weight
                matched_keywords.append(keyword)
        
        if score > 0:
            category_scores[category] = {
                'score': score,
                'keywords': matched_keywords,
                'confidence': min(score / 10.0, 1.0)
            }
    
    # Method 2: MediaPipe (optional)
    if use_mediapipe:
        try:
            classification_result = classifier.classify(description)
            if classification_result.classifications:
                top_classification = classification_result.classifications[0]
                if top_classification.categories:
                    top_category = top_classification.categories[0]
                    # MediaPipe result available but not directly used in final decision
        except Exception:
            pass  # Silently handle MediaPipe errors
    
    # Final decision
    if category_scores:
        best_category = max(category_scores.items(), key=lambda x: x[1]['score'])
        primary_category = best_category[0]
        confidence = best_category[1]['confidence']
        keywords_matched = best_category[1]['keywords']
        
        secondary_categories = [cat for cat, data in category_scores.items() 
                             if cat != primary_category and data['score'] >= 2]
        
        all_categories = [primary_category] + secondary_categories[:2]
        return all_categories, confidence, keywords_matched
    else:
        return ['Others'], 0.1, ['no_keywords_matched']

# ========================================================
# PART 5: Batch Processing Loop
# ========================================================

print(f"\nStarting batch processing...")
print(f"Processing {total_records:,} records in {total_batches} batches...")

current_file_index = 0
batches_in_current_file = 0
processed_records = 0

for batch_num in range(total_batches):
    # Check if we need to switch to next file
    if batches_in_current_file >= file_batch_counts[current_file_index]:
        current_file_index += 1
        batches_in_current_file = 0
        if current_file_index >= len(OUTPUT_FILES):
            break
    
    # Calculate batch boundaries
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, total_records)
    current_output_file = OUTPUT_FILES[current_file_index]
    
    print(f"\n--- Batch {batch_num + 1}/{total_batches} ---")
    print(f"Records {start_idx:,} to {end_idx-1:,}")
    print(f"Output: {current_output_file}")
    print(f"File {current_file_index + 1}/3, Batch {batches_in_current_file + 1}/{file_batch_counts[current_file_index]} in this file")
    
    # Extract batch
    df_batch = df.iloc[start_idx:end_idx].copy()
    
    # Process batch
    results = []
    for desc in tqdm(df_batch[col_name].astype(str).tolist(), 
                     desc=f"Batch {batch_num + 1}"):
        result = classify_business_description(desc)
        results.append(result)
    
    # Unpack results
    categories_list, confidences, keywords_matched = zip(*results)
    
    df_batch['amenity_categories'] = [', '.join(cats) for cats in categories_list]
    df_batch['classification_confidence'] = confidences
    df_batch['keywords_matched'] = [', '.join(kw) for kw in keywords_matched]
    
    # Save batch
    if batches_in_current_file == 0:
        # First batch in file - create with headers
        df_batch.to_csv(current_output_file, index=False, mode='w')
        print(f"Created: {current_output_file}")
    else:
        # Append without headers
        df_batch.to_csv(current_output_file, index=False, mode='a', header=False)
        print(f"Appended to: {current_output_file}")
    
    batches_in_current_file += 1
    processed_records += len(df_batch)
    
    # Quick batch stats
    batch_category_counts = (
        df_batch['amenity_categories']
        .str.split(', ')
        .explode()
        .value_counts()
    )
    
    retail_pct = (batch_category_counts.get('Retail_services', 0) / len(df_batch)) * 100
    others_pct = (batch_category_counts.get('Others', 0) / len(df_batch)) * 100
    
    print(f"Batch stats: Retail {retail_pct:.1f}%, Others {others_pct:.1f}%")
    print(f"Progress: {processed_records:,}/{total_records:,} ({processed_records/total_records*100:.1f}%)")
    
    # Memory cleanup
    del df_batch, results, categories_list, confidences, keywords_matched
    gc.collect()

# ========================================================
# PART 6: Final Summary
# ========================================================

print(f"\n" + "="*60)
print("BATCH PROCESSING COMPLETE!")
print("="*60)

# Check all output files
print(f"\nOutput File Summary:")
total_output_records = 0

for i, output_file in enumerate(OUTPUT_FILES):
    if os.path.exists(output_file):
        try:
            df_check = pd.read_csv(output_file)
            records_count = len(df_check)
            total_output_records += records_count
            print(f"{output_file}: {records_count:,} records")
            
            # Quick stats for each file
            category_counts = (
                df_check['amenity_categories']
                .str.split(', ')
                .explode()
                .value_counts()
            )
            retail_pct = (category_counts.get('Retail_services', 0) / len(df_check)) * 100
            others_pct = (category_counts.get('Others', 0) / len(df_check)) * 100
            print(f"  → Retail: {retail_pct:.1f}%, Others: {others_pct:.1f}%")
            
        except Exception as e:
            print(f"{output_file}: Error reading - {e}")
    else:
        print(f"{output_file}: Not created")

print(f"\nTotal output records: {total_output_records:,}")
print(f"Original records: {total_records:,}")
print(f"Success rate: {total_output_records/total_records*100:.1f}%")

if total_output_records == total_records:
    print(f"\n✅ SUCCESS! All records processed successfully.")
    print(f"📁 Data split across 3 files as requested.")
    print(f"🎯 Classification completed without Method 3 bias.")
else:
    print(f"\n⚠️  Warning: Record count mismatch.")

print(f"\n🏁 PROCESSING COMPLETE!")
print(f"Your classified data is ready in the v2 files.")

ACRA FULL DATASET CLASSIFICATION - BATCH PROCESSING
Batch size: 10,000 records
Output files: ['acra_classified_part1v2.csv', 'acra_classified_part2v2.csv', 'acra_classified_part3v2.csv']
Setting up MediaPipe Text Classifier...


I0000 00:00:1757173324.538273   15117 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1757173324.680654  370317 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Loading full ACRA dataset...
Full Dataset Shape: (951620, 15)
Final dataset size: 951,620 records
Total batches: 96
Batches per file: [32, 32, 32]
Records per file: [320000, 320000, 320000]

Starting batch processing...
Processing 951,620 records in 96 batches...

--- Batch 1/96 ---
Records 0 to 9,999
Output: acra_classified_part1v2.csv
File 1/3, Batch 1/32 in this file


Batch 1: 100%|██████████| 10000/10000 [04:28<00:00, 37.28it/s]


Created: acra_classified_part1v2.csv
Batch stats: Retail 52.3%, Others 10.6%
Progress: 10,000/951,620 (1.1%)

--- Batch 2/96 ---
Records 10,000 to 19,999
Output: acra_classified_part1v2.csv
File 1/3, Batch 2/32 in this file


...

Batch 26: 100%|██████████| 10000/10000 [26:40<00:00,  6.25it/s]   


Appended to: acra_classified_part1v2.csv
Batch stats: Retail 64.0%, Others 8.9%
Progress: 260,000/951,620 (27.3%)

--- Batch 27/96 ---
Records 260,000 to 269,999
Output: acra_classified_part1v2.csv
File 1/3, Batch 27/32 in this file


Batch 27: 100%|██████████| 10000/10000 [19:17<00:00,  8.64it/s]   


Appended to: acra_classified_part1v2.csv
Batch stats: Retail 67.7%, Others 7.3%
Progress: 270,000/951,620 (28.4%)

--- Batch 28/96 ---
Records 270,000 to 279,999
Output: acra_classified_part1v2.csv
File 1/3, Batch 28/32 in this file


Batch 28:   0%|          | 24/10000 [00:00<05:11, 32.00it/s]

In [2]:
import pandas as pd

# --- Read CSVs directly ---
df1 = pd.read_csv("acra_classified_part1.csv")
df2 = pd.read_csv("acra_classified_part2.csv")
df3 = pd.read_csv("acra_classified_part3.csv")

# --- Concatenate ---
merged = pd.concat([df1, df2, df3], ignore_index=True)

# --- Drop duplicates if needed ---
merged = merged.drop_duplicates()

# --- Save ---
merged.to_csv("acra_classified_merged.csv", index=False)

print(f"Merged file saved as acra_classified_merged.csv with {len(merged)} rows")


Merged file saved as acra_classified_merged.csv with 951620 rows
